In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

import os, cv2, time
import tqdm
import numpy as np
import math
from itertools import product
import matplotlib.pyplot as plt
from PIL import Image
import io
import pickle

PROJECT_PATH = 'D:/project/Aiffel/face detector'
DATA_PATH = os.path.join(PROJECT_PATH, 'widerface')
MODEL_PATH = os.path.join(PROJECT_PATH, 'checkpoints')
TRAIN_PT_PATH = os.path.join(PROJECT_PATH, 'data', 'train_data.pt')
VALID_PT_PATH = os.path.join(PROJECT_PATH, 'data', 'val_data.pt')
CHECKPOINT_PATH = os.path.join(PROJECT_PATH, 'checkpoints')

# 메모리 오류 수정을 위해 개별 파일들을 저장할 경로
TRAIN_PROCESSED_PATH = os.path.join(PROJECT_PATH, 'data', 'train_processed')
VALID_PROCESSED_PATH = os.path.join(PROJECT_PATH, 'data', 'val_processed')

DATASET_LEN = 12880
BATCH_SIZE = 8
IMAGE_WIDTH = 320
IMAGE_HEIGHT = 256
IMAGE_LABELS = ['background', 'face']

print(torch.__version__)

2.7.1+cu118


In [2]:
# 먼저 bounding box 파일을 분석해 봅시다. 분석에 필요한 코드를 함수 형태로 준비할게요.
# bounding box 파일의 한 줄을 입력받아, x0, y0, w, h 값을 반환하는 함수입니다.
def parse_box(data):
    x0 = int(data[0])
    y0 = int(data[1])
    w = int(data[2])
    h = int(data[3])
    return x0, y0, w, h

print('슝=3')

슝=3


In [3]:
# bounding box 파일 전체를 분석하는 함수입니다.
# 파일 이름과 bounding box 좌표 리스트를 튜플로 묶어 반환합니다.
def parse_widerface(file):
    infos = []
    with open(file) as fp:
        line = fp.readline()
        while line:
            n_object = int(fp.readline())
            boxes = []
            for i in range(n_object):
                box = fp.readline().split(' ')
                x0, y0, w, h = parse_box(box)
                if (w == 0) or (h == 0):
                    continue
                boxes.append([x0, y0, w, h])
            if n_object == 0:
                box = fp.readline().split(' ')
                x0, y0, w, h = parse_box(box)
                boxes.append([x0, y0, w, h])
            infos.append((line.strip(), boxes))
            line = fp.readline()
    return infos

print('슝=3')

슝=3


위 함수는 이미지별 bounding box 정보를 wider_face_train_bbx_gt.txt에서 파싱해서 리스트로 추출하는 것입니다.

이제 추출된 정보를 실제 이미지 정보와 결합합니다. bounding box 정보는 [x, y, w, h] 형태로 저장되어 있는데, [x_min, y_min, x_max, y_max] 형태의 꼭짓점 좌표 정보로 변환할 거예요.

이렇게 정보를 결합해야 나중에 학습에 사용하기 좋습니다.

In [4]:
# 이미지를 읽고, RGB로 변환한 후, CHW 형태의 Tensor로 바꾸는 함수입니다.
# image_file: 이미지 파일 경로
# return: (에러코드, 이미지 파일의 바이너리 문자열, CHW 형태의 Tensor)
# 에러코드: 0(성공), 1(실패)
# 이미지 파일의 바이너리 문자열: 이미지 파일을 바이너리 모드로 읽은 문자열
# CHW 형태의 Tensor: (채널, 높이, 너비) 형태의 Tensor
# 에러 발생 시, CHW 형태의 Tensor는 None을 반환합니다.
def process_image(image_file):
    try:
        with open(image_file, 'rb') as f:
            image_string = f.read()
            image_data = Image.open(io.BytesIO(image_string)).convert('RGB')
            image_data = torch.from_numpy(np.array(image_data)).permute(2, 0, 1)  # HWC to CHW
            return 0, image_string, image_data
    except Exception as e:
        return 1, image_string, None

print('슝=3')

슝=3


In [5]:
# bounding box 좌표를 VOC 형식으로 변환하는 함수입니다.
# file_name: 이미지 파일 이름
# boxes: bounding box 좌표 리스트 (x0, y0, w, h)
# image_data: CHW 형태의 Tensor
# return: 이미지 정보 딕셔너리
# 이미지 정보 딕셔너리: {'filename': 파일 이름, 'width': 너비, 'height': 높이, 'depth': 채널 수,
#                     'class': 클래스 리스트, 'xmin': xmin 리스트, 'ymin': ymin 리스트,
#                     'xmax': xmax 리스트, 'ymax': ymax 리스트, 'difficult': difficult 리스트}
# 클래스는 1(얼굴)로 고정, difficult는 0(쉬움)으로 고정
# xmin, ymin, xmax, ymax는 VOC 형식에 맞게 변환
# 에러 발생 시, 빈 딕셔너리를 반환합니다.
# VOC 형식: xmin, ymin, xmax, ymax (왼쪽 위 꼭짓점, 오른쪽 아래 꼭짓점)
# xywh 형식: x0, y0, w, h (왼쪽 위 꼭짓점, 너비, 높이)
def xywh_to_voc(file_name, boxes, image_data):
    shape = image_data.shape
    image_info = {}
    image_info['filename'] = file_name
    image_info['width'] = shape[1]
    image_info['height'] = shape[0]
    image_info['depth'] = 3

    difficult = []
    classes = []
    xmin, ymin, xmax, ymax = [], [], [], []

    for box in boxes:
        classes.append(1)
        difficult.append(0)
        xmin.append(box[0])
        ymin.append(box[1])
        xmax.append(box[0] + box[2])
        ymax.append(box[1] + box[3])
    image_info['class'] = classes
    image_info['xmin'] = xmin
    image_info['ymin'] = ymin
    image_info['xmax'] = xmax
    image_info['ymax'] = ymax
    image_info['difficult'] = difficult

    return image_info

print('슝=3')

슝=3


잘 결합된 데이터의 형태를 확인해 봅시다. 5개만 출력해 볼 거예요

In [6]:
# 준비한 함수를 테스트해 봅시다.
# wider_face_split/wider_face_train_bbx_gt.txt 파일을 분석해 봅시다.
# 앞에서 준비한 parse_widerface 함수를 사용합니다.
# bounding box 파일의 앞 5개 이미지만 테스트해 봅니다.
# bounding box 파일의 경로
file_path = os.path.join(DATA_PATH, 'wider_face_split', 'wider_face_train_bbx_gt.txt')
for i, info in enumerate(parse_widerface(file_path)):
    print('--------------------')
    image_file = os.path.join(DATA_PATH, 'WIDER_train', 'images', info[0])
    _, image_string, image_data = process_image(image_file)
    boxes = xywh_to_voc(image_file, info[1], image_data)
    print(boxes)
    if i > 3:
        break

--------------------
{'filename': 'D:/project/Aiffel/face detector\\widerface\\WIDER_train\\images\\0--Parade/0_Parade_marchingband_1_849.jpg', 'width': 1385, 'height': 3, 'depth': 3, 'class': [1], 'xmin': [449], 'ymin': [330], 'xmax': [571], 'ymax': [479], 'difficult': [0]}
--------------------
{'filename': 'D:/project/Aiffel/face detector\\widerface\\WIDER_train\\images\\0--Parade/0_Parade_Parade_0_904.jpg', 'width': 1432, 'height': 3, 'depth': 3, 'class': [1], 'xmin': [361], 'ymin': [98], 'xmax': [624], 'ymax': [437], 'difficult': [0]}
--------------------
{'filename': 'D:/project/Aiffel/face detector\\widerface\\WIDER_train\\images\\0--Parade/0_Parade_marchingband_1_799.jpg', 'width': 768, 'height': 3, 'depth': 3, 'class': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'xmin': [78, 78, 113, 134, 163, 201, 182, 245, 304, 328, 389, 406, 436, 522, 643, 653, 793, 535, 29, 3, 20], 'ymin': [221, 238, 212, 260, 250, 218, 266, 279, 265, 295, 281, 293, 290, 328, 320, 224, 

어떤가요? 이미지별로 boxes 리스트에 담긴 bounding box 정보가 확인되시나요? 이제 이 정보를 활용하여 텐서플로우 데이터셋을 생성해 봅시다.

## (2) 데이터셋 생성
TFRecord 만들기
오늘 다루게 될 대용량 데이터셋의 처리속도 향상을 위해서, 전처리 작업을 통해 TFRecord 데이터셋으로 변환할 필요가 있습니다. TFRecord란 TensorFlow만의 학습 데이터 저장 포맷으로, 이진(binary) 레코드의 시퀀스를 저장합니다. TFRecord 형태의 학습 데이터를 사용하여 모델 학습을 하면 학습 속도가 개선된다는 장점이 있습니다.

TFRecord는 여러 개의 tf.train.Example로 이루어져 있고, 한 개의 tf.train.Example은 여러 개의 tf.train.Feature로 이루어져 있습니다.

데이터의 단위를 이루는 tf.train.Example 인스턴스를 생성하는 메소드는 아래와 같습니다.

In [7]:
# 이미지 파일의 바이너리 문자열과 이미지 정보를 입력받아, 학습에 사용할 예제 딕셔너리를 반환하는 함수입니다.
# image_string: 이미지 파일의 바이너리 문자열
# image_infos: 이미지 정보 딕셔너리 리스트
# return: 예제 딕셔너리 (이미지 파일 이름, 너비, 높이, 채널 수, 클래스 리스트, xmin 리스트, ymin 리스트,
#                     xmax 리스트, ymax 리스트, 이미지 파일의 바이너리 문자열) 형태 의 딕셔너리 
import io

def make_example(image_string, image_infos):
    for info in image_infos:
        filename = info['filename']
        width = info['width']
        height = info['height']
        depth = info['depth']
        classes = info['class']
        xmin = info['xmin']
        ymin = info['ymin']
        xmax = info['xmax']
        ymax = info['ymax']

    # 이미지 데이터를 numpy 배열로 변환
    image_data = np.frombuffer(image_string, dtype=np.uint8)
    image_data = Image.open(io.BytesIO(image_data)).convert('RGB')
    image_data = np.array(image_data)

    # 데이터를 dict 형태로 저장
    example = {
        'filename': filename,
        'height': height,
        'width': width,
        'classes': classes,
        'xmin': xmin,
        'ymin': ymin,
        'xmax': xmax,
        'ymax': ymax,
        'image_raw': image_data
    }

    return example

print('슝=3')

슝=3


이제 전처리를 위해 필요한 함수들이 어느 정도 갖추어졌습니다. 데이터셋의 이미지 파일, 그리고 bounding box 정보를 모아 위의 make_example 메소드를 통해 만든 example을 serialize하여 TFRecord 파일로 생성하게 됩니다.

TFRecord에 대한 자세한 사항은 아래를 참고하세요.

TFRecord
⚠️ 주의!! 아래 코드를 실행하면 데이터셋이 담긴 .pt 파일이 경로에 저장됩니다!
약 2시간 가량이 소요되니, 미리 저장된 파일을 활용해주세요!

In [8]:
# 기존 코드 주석 처리
# # 준비한 함수를 사용하여, 학습과 검증에 사용할 데이터셋을 만듭니다.
# # 학습 데이터셋은 wider_face_split/wider_face_train_bbx_gt.txt 파일을 사용하고,
# # 검증 데이터셋은 wider_face_split/wider_face_val_bbx_gt.txt 파일을 사용합니다.
# # 각각의 데이터셋은 .pt 파일로 저장합니다.
# # (2) 데이터셋 생성
#
# import torch
# import os
# import tqdm
# from PIL import Image
#
# for split in ['train', 'val']:
#     if split == 'train':
#         output_file = TRAIN_PT_PATH
#         anno_txt = 'wider_face_train_bbx_gt.txt'
#         file_path = 'WIDER_train'
#     else:
#         output_file = VALID_PT_PATH
#         anno_txt = 'wider_face_val_bbx_gt.txt'
#         file_path = 'WIDER_val'
#
#     dataset = []  # 데이터를 저장할 리스트
#
#     for info in tqdm.tqdm(parse_widerface(os.path.join(DATA_PATH, 'wider_face_split', anno_txt))):
#         image_file = os.path.join(DATA_PATH, file_path, 'images', info[0])
#         error, image_string, image_data = process_image(image_file)
#         boxes = xywh_to_voc(image_file, info[1], image_data)
#
#         if not error:
#             example = make_example(image_string, [boxes])
#             dataset.append(example)
#
#     # dataset을 .pt 파일로 저장
#     torch.save(dataset, output_file)
#
# print('슝=3')

# 메모리 오류 해결을 위한 수정 코드
import torch
import os
import tqdm
from PIL import Image

for split in ['train', 'val']:
    if split == 'train':
        output_dir = TRAIN_PROCESSED_PATH
        anno_txt = 'wider_face_train_bbx_gt.txt'
        file_path = 'WIDER_train'
    else:
        output_dir = VALID_PROCESSED_PATH
        anno_txt = 'wider_face_val_bbx_gt.txt'
        file_path = 'WIDER_val'

# 개별 파일들을 저장할 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    print(f'{output_dir} 디렉토리를 생성했습니다.')

# tqdm의 enumerate를 사용하여 인덱스와 데이터를 함께 가져옵니다.
    data_source = parse_widerface(os.path.join(DATA_PATH, 'wider_face_split', anno_txt))
    for i, info in enumerate(tqdm.tqdm(data_source)):
        image_file = os.path.join(DATA_PATH, file_path, 'images', info[0])
        error, image_string, image_data = process_image(image_file)
        boxes = xywh_to_voc(image_file, info[1], image_data)

        if not error:
            example = make_example(image_string, [boxes])
# 개별 파일로 즉시 저장
            torch.save(example, os.path.join(output_dir, f'{i}.pt'))

print('슝=3')

D:/project/Aiffel/face detector\data\train_processed 디렉토리를 생성했습니다.


  3%|▎         | 342/12880 [00:13<08:13, 25.40it/s]


KeyboardInterrupt: 

MemoryError: 가 발생했다.

생성된 .pt 파일을 확인해 봅시다.

In [16]:
!ls /content/drive/MyDrive/data/face_detector/data

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# # 파일이 잘 저장되었는지 확인해볼까요??
# # 저장된 파일 로드
# train_data = torch.load(TRAIN_PT_PATH)
# val_data = torch.load(VALID_PT_PATH)

# # 데이터 개수 확인
# print(f"Train 데이터 개수: {len(train_data)}")
# print(f"Validation 데이터 개수: {len(val_data)}")

# # 샘플 데이터 확인
# sample = train_data[0]
# print(f"샘플 데이터 타입: {type(sample)}")
# print(f"샘플 데이터 내용: {sample}")

#   메모리 오류 해결을 위한 수정된 데이터 로딩 코드
import torch
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader

class ProcessedDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
#   디렉토리에서 .pt 파일 목록을 가져옵니다.
        self.file_list = [f for f in os.listdir(data_dir) if f.endswith('.pt')]
#   파일 이름을 정수형으로 변환하여 정렬합니다. (예: '1.pt', '10.pt', '2.pt' -> '1.pt', '2.pt', '10.pt')
        self.file_list.sort(key=lambda x: int(x.split('.')[0]))

    def __len__(self):
#   데이터셋의 전체 길이는 파일의 개수입니다.
        return len(self.file_list)

    def __getitem__(self, idx):
#   주어진 인덱스(idx)에 해당하는 파일을 로드합니다.
        file_path = os.path.join(self.data_dir, self.file_list[idx])
        sample = torch.load(file_path, weights_only=False)  # PyTorch 버전 변경으로 인해 weights_only=False 옵션 추가
        return sample

#   파일이 잘 저장되었는지 확인
#   저장된 개별 파일들을 로드하기 위한 Dataset 객체 생성
train_dataset = ProcessedDataset(TRAIN_PROCESSED_PATH)
val_dataset = ProcessedDataset(VALID_PROCESSED_PATH)

#   데이터 개수 확인
print(f"Train 데이터 개수: {len(train_dataset)}")
print(f"Validation 데이터 개수: {len(val_dataset)}")

#   샘플 데이터 확인 (첫 번째 데이터)
if len(train_dataset) > 0:
    sample = train_dataset[0]
    print(f"샘플 데이터 타입: {type(sample)}")
    #   샘플 데이터의 모든 키와 값의 타입을 출력해봅니다.   
    for key, value in sample.items():
        if isinstance(value, np.ndarray):
            print(f"  - {key}: {type(value)}, shape: {value.shape}")
        else:
            print(f"  - {key}: {type(value)}")
else:
    print("Train 데이터셋이 비어있습니다.")

# DataLoader 사용 예시 (실제 학습 시 이렇게 사용합니다)
#   train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

Train 데이터 개수: 12880
Validation 데이터 개수: 3226
샘플 데이터 타입: <class 'dict'>
  - filename: <class 'str'>
  - height: <class 'int'>
  - width: <class 'int'>
  - classes: <class 'list'>
  - xmin: <class 'list'>
  - ymin: <class 'list'>
  - xmax: <class 'list'>
  - ymax: <class 'list'>
  - image_raw: <class 'numpy.ndarray'>, shape: (1385, 1024, 3)


3. 모델 구현
(1) Default boxes
SSD의 Default box
SSD 모델의 가장 중요한 특징 중 하나는 Default box를 필요로 한다는 점입니다. 앞서 이야기한 대로 Default box란, object가 존재할 만한 다양한 크기의 box의 좌표 및 클래스 정보를 일정 개수만큼 미리 고정해 둔 것입니다. 흔히 anchor box, prior box라고 부르지요. SSD의 Default box가 약간 다른 점은 여러 층의 feature map에서 box를 만들어 낸다는 점이에요. 층 수 만큼 box 수도 많아지고, 층마다 box의 크기도 다양하게 되죠. ground truth에 해당하는 bounding box와의 IoU를 계산하여 일정 크기(0.5) 이상 겹치는 default box를 선택하는 방식이 RCNN 계열의 sliding window 방식보다 훨씬 속도가 빠르면서도 그와 유사한 정도의 정확도를 얻을 수 있습니다.

https://manalelaidouni.github.io/Single%20shot%20object%20detection.html

[Default box]

이번 프로젝트에서 사용할 default box 정보를 전역 변수로 만들어 둡니다.

In [10]:
BOX_MIN_SIZES = [[10, 16, 24], [32, 48], [64, 96], [128, 192, 256]]
BOX_STEPS = [8, 16, 32, 64]

print('슝=3')

슝=3


https://towardsdatascience.com/understanding-ssd-multibox-real-time-object-detection-in-deep-learning-495ef744fab
[SSD default boxes at 8x8 and 4x4 feature maps]

위 그림에서 보는 것처럼, default box를 생성하기 위해서는 먼저 기준이 되는 feature map을 먼저 생성합니다. 그림에서는 8 X 8, 4 X 4의 예가 나오지만, 우리의 프로젝트에서는 아래와 같이 4가지 유형의 feature map을 생성하게 됩니다.

In [11]:
image_sizes = (IMAGE_HEIGHT, IMAGE_WIDTH)
min_sizes = BOX_MIN_SIZES
steps= BOX_STEPS

feature_maps = [
    [math.ceil(image_sizes[0] / step), math.ceil(image_sizes[1] / step)]
    for step in steps
]
feature_maps

[[32, 40], [16, 20], [8, 10], [4, 5]]

In [ ]:
이제 feature map별로 순회를 하면서 default box 를 생성해 보겠습니다.

In [12]:
boxes = []
for k, f in enumerate(feature_maps):
    for i, j in product(range(f[0]), range(f[1])):
        for min_size in min_sizes[k]:
            s_kx = min_size / image_sizes[1]
            s_ky = min_size / image_sizes[0]
            cx = (j + 0.5) * steps[k] / image_sizes[1]
            cy = (i + 0.5) * steps[k] / image_sizes[0]
            boxes += [cx, cy, s_kx, s_ky]

len(boxes)

18800

생성된 boxes에는 default box 정보가 구분없이 나열되어 있으므로 4개씩 재배열 시켜줄게요.

In [13]:
pretty_boxes = np.asarray(boxes).reshape([-1, 4])
print(pretty_boxes.shape)
print(pretty_boxes)

(4700, 4)
[[0.0125    0.015625  0.03125   0.0390625]
 [0.0125    0.015625  0.05      0.0625   ]
 [0.0125    0.015625  0.075     0.09375  ]
 ...
 [0.9       0.875     0.4       0.5      ]
 [0.9       0.875     0.6       0.75     ]
 [0.9       0.875     0.8       1.       ]]


모두 4700개의 default box가 만들어 졌네요. feature_maps와 min_sizes로부터 40x32x3 + 20x16x2 + 10x8x2 + 5x4x3 개가 생성되었다는 걸 확인할 수 있습니다.

지금까지 만들었던, feature map을 만들고 그에 연결된 default box를 생성하는 코드를 사용하기 편리하도록 함수로 정의해 둡시다.

In [14]:
def default_box():
    image_sizes = (IMAGE_HEIGHT, IMAGE_WIDTH)
    min_sizes = BOX_MIN_SIZES
    steps= BOX_STEPS
    feature_maps = [
        [math.ceil(image_sizes[0] / step), math.ceil(image_sizes[1] / step)]
        for step in steps
    ]
    boxes = []
    for k, f in enumerate(feature_maps):
        for i, j in product(range(f[0]), range(f[1])):
            for min_size in min_sizes[k]:
                s_kx = min_size / image_sizes[1]
                s_ky = min_size / image_sizes[0]
                cx = (j + 0.5) * steps[k] / image_sizes[1]
                cy = (i + 0.5) * steps[k] / image_sizes[0]
                boxes += [cx, cy, s_kx, s_ky]
    boxes = np.asarray(boxes).reshape([-1, 4])
    return boxes

print('슝=3')

슝=3


# (2) SSD
SSD model 빌드하기
그럼 본격적으로 SSD 모델을 생성해 보겠습니다. 우선은 SSD 모델 내부에서 사용하는 레이어들을 아래와 같이 생성합니다.

일반적으로 많이 쓰이는 Convolution 블록, Depthwise Convolution 블록, 그리고 skip connection으로 쓰일 Branch 블록을 준비합니다.

In [15]:
def _conv_block(inputs, filters, kernel=(3, 3), strides=(1, 1)):
    block_id = id(inputs)

    if strides == (2, 2):
        x = F.pad(inputs, (1, 1, 1, 1), mode='constant', value=0)  # ZeroPadding2D
        x = nn.Conv2d(inputs.size(1), filters, kernel_size=kernel, stride=strides, padding=0, bias=False)(x)
    else:
        x = nn.Conv2d(inputs.size(1), filters, kernel_size=kernel, stride=strides, padding='same', bias=False)(inputs)

    x = nn.BatchNorm2d(filters)(x)
    return F.relu(x)

print('슝=3')

슝=3


In [16]:
class DepthwiseConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, strides=(1, 1)):
        super(DepthwiseConvBlock, self).__init__()
        self.strides = strides

        if strides != (1, 1):
            self.pad = nn.ZeroPad2d((1, 1, 1, 1))
        else:
            self.pad = nn.Identity()

        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=strides, padding=0 if strides != (1, 1) else 1, groups=in_channels, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.pad(x)
        x = self.depthwise(x)
        x = F.relu(self.bn1(x))
        x = self.pointwise(x)
        return F.relu(self.bn2(x))

print('슝=3')

슝=3


In [17]:
class BranchBlock(nn.Module):
    def __init__(self, in_channels, filters):
        super(BranchBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(filters, filters, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, filters * 2, kernel_size=3, padding=1)

    def forward(self, x):
        x1 = F.leaky_relu(self.conv1(x))
        x1 = self.conv2(x1)
        x2 = self.conv3(x)
        x = torch.cat([x1, x2], dim=1)
        return F.relu(x)

print('슝=3')

슝=3


여러 블록을 쌓아 모델을 만든 후, 중간중간 Branch 부분에 헤드(head) 라고 불리는 Convolution 레이어를 붙일 거예요. 하나의 헤드에 Convolution 레이어 두 개가 필요합니다. 하나는 confidence를 예측하기 위해 사용하고 다른 하나는 location을 예측하기 위해 사용해요.

Branch마다 헤드가 연결되어 있기 때문에 모델의 중간 레이어에서도 예측을 위한 정보를 가져올 수 있게 되었습니다.

In [18]:
class HeadBlock(nn.Module):
    def __init__(self, in_channels, filters):
        super(HeadBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, filters, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        return self.conv(x)

print('슝=3')

슝=3


In [19]:
def _compute_heads(inputs, num_class, num_cell):
    conf = HeadBlock(inputs.size(1), num_cell * num_class)(inputs)
    conf = conf.permute(0, 2, 3, 1).contiguous().view(inputs.size(0), -1, num_class)

    loc = HeadBlock(inputs.size(1), num_cell * 4)(inputs)
    loc = loc.permute(0, 2, 3, 1).contiguous().view(inputs.size(0), -1, 4)

    return conf, loc

print('슝=3')

슝=3


In [214]:
class SsdModel(nn.Module):
    def __init__(self, image_height, image_width, image_labels):
        super(SsdModel, self).__init__()

        self.base_channel = 16
        self.num_cells = [3, 2, 2, 3]
        self.num_class = len(image_labels)

        self.conv_blocks = nn.ModuleList([
            DepthwiseConvBlock(3, self.base_channel * 4, strides=(1, 1)),
            BranchBlock(self.base_channel * 4, self.base_channel * 8)
        ])

        branch_block_output_channels = (self.base_channel * 8) + (self.base_channel * 8 * 2)
# 384

        # BranchBlock(in=256, filters=16)의 출력은 16 + 16*2 = 48 채널입니다.
        output_of_second_branch = self.base_channel + self.base_channel * 2 # 48

        self.layers = nn.ModuleList([
            DepthwiseConvBlock(branch_block_output_channels, self.base_channel * 16,
strides=(2, 2)),
            BranchBlock(self.base_channel * 16, self.base_channel),
            # self.layers[1]의 출력(48)을 입력으로 받도록 수정합니다.
            DepthwiseConvBlock(output_of_second_branch, self.base_channel * 16, strides=(2, 2)),
            BranchBlock(self.base_channel * 16, self.base_channel)
        ])

    def forward(self, x):
        x1 = self.conv_blocks[0](x)
        x2 = self.conv_blocks[1](x1)

        x3 = self.layers[0](x2)
        x3 = self.layers[1](x3)

        x4 = self.layers[2](x3)
        x4 = self.layers[3](x4)

        extra_layers = [x1, x2, x3, x4]

        confs, locs = [], []

        for layer, num_cell in zip(extra_layers, self.num_cells):
            conf, loc = _compute_heads(layer, self.num_class, num_cell)
            confs.append(conf)
            locs.append(loc)

        confs = torch.cat(confs, dim=1)
        locs = torch.cat(locs, dim=1)

        predictions = torch.cat([locs, confs], dim=2)

        return predictions

print("SsdModel의 두 번째 채널 불일치 문제를 수정했습니다.")

SsdModel의 두 번째 채널 불일치 문제를 수정했습니다.


레이어들이 준비되었습니다. 이제 본격적으로 SSD model을 준비해 보겠습니다.

슝=3


준비한 모델을 생성해 봅시다.

In [21]:
model = SsdModel(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_LABELS)
print("the number of model layers: ", len(list(model.modules())))
print(model)

print('슝=3')

the number of model layers:  33
SsdModel(
  (conv_blocks): ModuleList(
    (0): DepthwiseConvBlock(
      (pad): Identity()
      (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
      (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pointwise): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BranchBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (layers): ModuleList(
    (0): DepthwiseConvBlock(
      (pad): ZeroPad2d((1, 1, 1, 1))
      (depthwise): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), groups=128, bias=False)
      (bn1): BatchNorm2d(128, eps=1e

33층의 SSD 모델이 완성되었습니다!

이제 자유롭게 모델 사이즈를 줄여도 보고, 늘려도 보면서 여러분들만의 모델의 model.summary( )를 출력해보세요. (정해진 답은 없습니다 ㅎㅎ)

In [22]:
# _compute_heads 함수 구현
class _compute_heads(nn.Module):
    def __init__(self, in_channels, num_cell, out_channels):
        super(_compute_heads, self).__init__()
        self.conv = nn.Conv2d(in_channels, num_cell * out_channels, kernel_size=3, padding=1)
        self.num_cell = num_cell
        self.out_channels = out_channels

    def forward(self, x):
        out = self.conv(x)
        out = out.permute(0, 2, 3, 1).contiguous()
        out = out.view(out.size(0), -1, self.out_channels)
        return out

class YourSsdModel(nn.Module):
    def __init__(self, num_classes=2, input_shape=(3, 300, 300), num_cells=[4, 6, 6, 6]):
        super(YourSsdModel, self).__init__()

        self.num_classes = num_classes
        self.num_cells = num_cells

        self.conv1 = nn.Conv2d(input_shape[0], 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conf_layers = nn.ModuleList()
        self.loc_layers = nn.ModuleList()

        for num_cell in num_cells:
            self.conf_layers.append(_compute_heads(256, num_cell, self.num_classes))  # 클래스 예측
            self.loc_layers.append(_compute_heads(256, num_cell, 4))  # 바운딩 박스 좌표 예측

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)

        confs = []
        locs = []

        for conf_layer, loc_layer in zip(self.conf_layers, self.loc_layers):
            confs.append(conf_layer(x))
            locs.append(loc_layer(x))

        confs = torch.cat(confs, dim=1)  # (batch, total_boxes, num_classes)
        locs = torch.cat(locs, dim=1)  # (batch, total_boxes, 4)

        return confs, locs

print('슝=3')

슝=3


In [23]:
# 모델 불러오기,
mymodel = YourSsdModel()
print(mymodel)

YourSsdModel(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conf_layers): ModuleList(
    (0): _compute_heads(
      (conv): Conv2d(256, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-3): 3 x _compute_heads(
      (c

## 4. 모델 학습
### (1) Augmentation, jaccard 적용
#### Augmentation
이전 스텝에서 모델까지 구현하였습니다. 그러나 본격적으로 학습을 진행하기 전에 아직 해야할 것이 몇 가지 더 남아있습니다.

이전 스텝에서 구성한 TFRecord 형태의 데이터셋은 아직 Data augmentation이 적용되지 않았습니다. Object detection에서 사용하는 다양한 augmentation 기법을 적용해 주면 성능이 조금 더 향상될 수도 있습니다.

아래는 augmentation을 위해 tf.data.TFRecordDataset.map() 내에서 호출할 메소드들입니다.

- _crop
- _pad_to_square
- _resize
- _flip
- _distort

Image Classification에서 사용하던 함수와는 비교가 안 될 정도로 복잡한 함수들입니다. 이미지가 변경되면서 box의 위치나 크기도 함께 변경되어야 하기 때문에 함수가 복잡해질 수 밖에 없어요.

이미지를 _crop했을 때 box도 함께 잘릴 수가 있어요. 함께 신경써서 잘라줍니다.

In [130]:
def _crop(img, labels, max_loop=250):
    # img는 (채널, 높이, 너비) 형태의 텐서입니다.
    shape = img.shape

    def matrix_iof(a, b):
        lt = torch.maximum(a[:, None, :2], b[:, :2])
        rb = torch.minimum(a[:, None, 2:], b[:, 2:])
        area_i = torch.prod(rb - lt, dim=2) * (lt < rb).all(dim=2).float()
        area_a = torch.prod(a[:, 2:] - a[:, :2], dim=1)
        return area_i / torch.maximum(area_a[:, None], torch.tensor(1.0))

    for _ in range(max_loop):
        pre_scale = torch.tensor([0.3, 0.45, 0.6, 0.8, 1.0], dtype=torch.float32)
        scale = pre_scale[torch.randint(0, 5, (1,))]

        # shape[0]이 아닌 shape[1](높이)과 shape[2](너비)를 사용하도록 수정
        short_side = min(shape[1], shape[2])
        h = w = int(scale * short_side)

        # 이미지가 너무 작아 자를 수 없는 경우를 방지
        if shape[1] < h or shape[2] < w:
            continue

        h_offset = torch.randint(0, shape[1] - h + 1, (1,)).item()
        w_offset = torch.randint(0, shape[2] - w + 1, (1,)).item()

        roi = torch.tensor([w_offset, h_offset, w_offset + w, h_offset + h],
dtype=torch.float32)

        value = matrix_iof(labels[:, :4], roi[None, :])
        if torch.any(value >= 1):
            centers = (labels[:, :2] + labels[:, 2:4]) / 2
            mask_a = (roi[:2] < centers).all(dim=1) & (centers < roi[2:]).all(dim=1)
            if mask_a.any():
                # (채널, 높이, 너비)에 맞는 올바른 슬라이싱으로 수정
                img_t = img[:, h_offset:h_offset + h, w_offset:w_offset + w]

                labels_t = labels[mask_a]
                labels_t[:, :4] -= torch.tensor([w_offset, h_offset, w_offset, h_offset],
dtype=torch.float32)
                return img_t, labels_t

    return img, labels

print("_crop 함수의 텐서 슬라이싱 방식을 수정했습니다.")

_crop 함수의 텐서 슬라이싱 방식을 수정했습니다.


_resize나 _flip도 box에 영향을 끼칩니다.

In [208]:
def _resize(img, labels):
    # --- 최종 디버깅 코드 ---
    # _resize 함수에 들어온 텐서의 형태를 무조건 출력합니다.
    print(f"DEBUG: _resize 함수에 들어온 텐서 형태: {img.shape}")

    # 4D 텐서 방어 코드
    if img.dim() == 4:
        img = img.squeeze(0)

    h_f, w_f = img.shape[1:3]

    locs = torch.stack([labels[:, 0] / w_f, labels[:, 1] / h_f,
                        labels[:, 2] / w_f, labels[:, 3] / h_f], dim=1)

    locs = torch.clamp(locs, 0, 1.0)
    labels = torch.cat([locs, labels[:, 4].unsqueeze(1)], dim=1)

    resize_case = torch.randint(0, 4, (1,)).item()

    resize_methods = [
        T.Resize((IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=T.InterpolationMode.BICUBIC),
        T.Resize((IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=T.InterpolationMode.NEAREST),
        T.Resize((IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=T.InterpolationMode.LANCZOS),
        T.Resize((IMAGE_HEIGHT, IMAGE_WIDTH), interpolation=T.InterpolationMode.BILINEAR)
    ]

    img = resize_methods[resize_case](img)

    return img, labels

print("_resize 함수에 최종 디버깅 코드를 삽입했습니다.")

_resize 함수에 최종 디버깅 코드를 삽입했습니다.


In [188]:
def _flip(img, labels):
    flip_case = torch.randint(0, 2, (1,)).item()

    if flip_case == 0:
        img = torch.flip(img, dims=[2])

        labels = torch.stack([
            1 - labels[:, 2], labels[:, 1],
            1 - labels[:, 0], labels[:, 3],
            labels[:, 4]
        ], dim=1)

    return img, labels

print('슝=3')

슝=3


_pad_to_square는 이미지가 정사각형이 되도록 이미지 오른쪽이나 아래 방향으로 평균 색상 영역을 추가해주는 작업입니다.

다행히도 이미지 왼쪽이나 위쪽으로는 변화가 없기 때문에 box 정보는 변하지 않습니다.

In [189]:
def _pad_to_square(img):
    h, w = img.shape[1:3]

    if h > w:
        pad = (h - w) // 2
        img = F.pad(img, (pad, h - w - pad, 0, 0), value=img.mean())
    elif w > h:
        pad = (w - h) // 2
        img = F.pad(img, (0, 0, pad, w - h - pad), value=img.mean())

    return img

print('슝=3')

슝=3


이미지 색상 값만 바꿔주는 _distort에서도 box 정보를 바꿀 필요는 없겠네요!

In [190]:
def _distort(img):
    img = T.ColorJitter(brightness=0.4, contrast=0.5, saturation=0.5, hue=0.1)(img)
    return img

print('슝=3')

슝=3


### Default box 적용
SSD 모델의 특이점 중 하나가 default box를 사용한다는 점을 이미 앞에서 설명하였습니다. default box 정보는 데이터셋에 반영되어야 합니다. 아래 메소드들은 default box와 bounding box 사이의 IoU, 다른 말로 자카드 유사도(자카드 지수, jaccard index)를 측정하기 위한 것입니다.

자카드 유사도는 두 집합을 통해 유사도를 측정하는 방식 중 하나로, 두 집합의 교집합을 두 집합의 합집합으로 나눕니다. 따라서 자카드 유사도는 0과 1 사이의 값을 가지며, 두 집합의 합집합과 교집합이 서로 비슷하면 1에 근접하다는 뜻입니다.

자카드 유사도의 공식은 아래와 같습니다.

자카드 유사도

이제 이 메소드를 활용해 어떻게 데이터셋을 추가로 가공하는지 살펴봅시다.

In [191]:
def _intersect(box_a, box_b):
    A = box_a.shape[0]
    B = box_b.shape[0]

    max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2))
    min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))

    inter = torch.clamp((max_xy - min_xy), min=0)

    return inter[:, :, 0] * inter[:, :, 1]

print('슝=3')

슝=3


In [192]:
def _jaccard(box_a, box_b):
    inter = _intersect(box_a, box_b)

    area_a = ((box_a[:, 2] - box_a[:, 0]) * (box_a[:, 3] - box_a[:, 1])).unsqueeze(1).expand_as(inter)
    area_b = ((box_b[:, 2] - box_b[:, 0]) * (box_b[:, 3] - box_b[:, 1])).unsqueeze(0).expand_as(inter)

    union = area_a + area_b - inter

    return inter / union

print('슝=3')

슝=3


자카드 유사도를 계산하는 메소드가 준비되었습니다. 아래 encode_tf는 이를 이용해서 TFRecord 데이터셋의 라벨을 가공하는 메소드입니다. 내용을 정리하면 다음과 같습니다.

- jaccard 메소드를 이용해 label의 ground truth bbox와 가장 overlap 비율이 높은 matched box를 구한다.
- _encode_bbox 메소드를 통해 bbox의 scale을 동일하게 보정한다.
- 전체 default box에 대해 일정 threshold 이상 overlap되는 ground truth bounding box 존재 여부(positive/negative)를 concat하여 새로운 label로 업데이트한다.

In [193]:
def _encode_bbox(matched, boxes, variances=[0.1, 0.2]):
    g_cxcy = (matched[:, :2] + matched[:, 2:]) / 2 - boxes[:, :2]
    g_cxcy /= (variances[0] * boxes[:, 2:])

    g_wh = (matched[:, 2:] - matched[:, :2]) / boxes[:, 2:]
    g_wh = torch.log(g_wh) / variances[1]

    g_wh = torch.where(torch.isinf(g_wh), torch.zeros_like(g_wh), g_wh)

    return torch.cat([g_cxcy, g_wh], dim=1)

print('슝=3')

슝=3


In [194]:
def encode_pt(labels, boxes):
    match_threshold = 0.45

    boxes = boxes.float()
    bbox = labels[:, :4]
    conf = labels[:, -1]

    overlaps = _jaccard(bbox, boxes)

    best_box_overlap, best_box_idx = overlaps.max(dim=1)
    best_truth_overlap, best_truth_idx = overlaps.max(dim=0)

    best_truth_overlap[best_box_idx] = 2.0
    best_truth_idx[best_box_idx] = torch.arange(best_box_idx.size(0))

    matches_bbox = bbox[best_truth_idx]
    loc_t = _encode_bbox(matches_bbox, boxes)

    conf_t = conf[best_truth_idx]
    conf_t[best_truth_overlap < match_threshold] = 0

    return torch.cat([loc_t, conf_t.unsqueeze(1)], dim=1)

print('슝=3')

슝=3


### load_dataset
위에서 구현한 두가지 메소드를 이전 스텝에서 생성한 tfrecord 데이터셋에 적용하여 SSD 학습을 위한 데이터셋을 생성하는 최종 메소드인 load_dataset 을 구현합니다.

- _transform_data : augmemtation과 label을 encoding 하여 기존의 dataset을 변환하는 메소드
- _parse_tfrecord : TFRecord 에 _transform_data를 적용하는 함수 클로저 생성
- load_tfrecord_dataset : tf.data.TFRecordDataset.map()에 _parse_tfrecord을 적용하는 실제 데이터셋 변환 메인 메소드
- load_dataset : load_tfrecord_dataset을 통해 train, validation 데이터셋을 생성하는 최종 메소드

In [217]:
def _transform_data(train, boxes):
    def transform_data(img, labels):
        img = img.float()

        if train:
            # _crop은 비활성화 상태입니다.
            # img, labels = _crop(img, labels)

            # --- 강제 차원 고정 1 ---
            if img.dim() == 4: img = img.squeeze(0)
            img = _pad_to_square(img)

            # --- 강제 차원 고정 2 ---
            if img.dim() == 4: img = img.squeeze(0)

        # _resize 함수에도 방어 코드가 있지만, 중복으로 확인합니다.
        img, labels = _resize(img, labels)

        # --- 강제 차원 고정 3 ---
        if img.dim() == 4: img = img.squeeze(0)

        if train:
            img, labels = _flip(img, labels)
            # --- 강제 차원 고정 4 ---
            if img.dim() == 4: img = img.squeeze(0)

            img = _distort(img)
            # --- 강제 차원 고정 5 ---
            if img.dim() == 4: img = img.squeeze(0)

        labels = encode_pt(labels, boxes)
        img = img / 255.0

        return img, labels

    return transform_data

print("_transform_data 함수에 강제 차원 고정 코드를 추가했습니다.")

_transform_data 함수에 강제 차원 고정 코드를 추가했습니다.


In [196]:
def _parse_pt(train, boxes):
    def parse_pt(pt):
        example = pickle.loads(pt.numpy())

        img = torch.tensor(np.frombuffer(example['image_raw'], dtype=np.uint8).reshape(3, example['height'], example['width']))

        labels = torch.tensor(np.stack([
            example['x_mins'],
            example['y_mins'],
            example['x_maxes'],
            example['y_maxes'],
            example['classes']], axis=1), dtype=torch.float32)

        img, labels = _transform_data(train, boxes)(img, labels)

        return img, labels

    return parse_pt

In [197]:
def load_pt_dataset(pt_name, train=True, boxes=None, buffer_size=64):
    with open(pt_name, 'rb') as f:
        raw_data = f.read()

    raw_array = np.frombuffer(raw_data, dtype=np.uint8)
    raw_dataset = [torch.tensor(raw_array, dtype=torch.uint8)]

    if train:
        np.random.shuffle(raw_dataset)

    return raw_dataset

print('슝=3')

슝=3


In [198]:
def load_dataset(boxes, train=True, buffer_size=64):
    pt_name = TRAIN_PT_PATH if train else VALID_PT_PATH

    return load_pt_dataset(pt_name, train, boxes, buffer_size)

print('슝=3')

슝=3


상당히 긴 데이터 준비 과정이 끝났습니다.

## (2) Training
### Learning rate scheduler
본격적으로 train에 들어가기 전에 2가지 더 준비해야 할 게 있습니다. 그중 하나는 Learning rate scheduler 입니다. 이번에는 초기시점에 WarmUp부분을 도입해 learning rate가 천천히 증가할 수 있도록 학습 스텝에 따라 다른 Learning Rate이 적용될 수 있도록 하겠습니다. PiecewiseConstantWarmUpDecay정도의 이름으로 만들어 사용할게요.

In [199]:
class PiecewiseConstantWarmUpDecay:
    def __init__(self, boundaries, values, warmup_steps, min_lr):
        if len(boundaries) != len(values) - 1:
            raise ValueError("The length of boundaries should be 1 less than the length of values")

        self.boundaries = boundaries
        self.values = values
        self.warmup_steps = warmup_steps
        self.min_lr = min_lr

    def __call__(self, step):
        if step <= self.warmup_steps:
            return self.min_lr + step * (self.values[0] - self.min_lr) / self.warmup_steps

        for i, boundary in enumerate(self.boundaries):
            if step <= boundary:
                return self.values[i]

        return self.values[-1]

print('슝=3')

슝=3


PiecewiseConstantWarmUpDecay를 편하게 사용할 수 있도록 함수를 만들어 둡니다.

In [200]:
def MultiStepWarmUpLR(initial_learning_rate, lr_steps, lr_rate, warmup_steps=0, min_lr=0):
    assert warmup_steps <= lr_steps[0]
    assert min_lr <= initial_learning_rate

    lr_steps_value = [initial_learning_rate]
    for _ in range(len(lr_steps)):
        lr_steps_value.append(lr_steps_value[-1] * lr_rate)

    return PiecewiseConstantWarmUpDecay(boundaries=lr_steps, values=lr_steps_value, warmup_steps=warmup_steps, min_lr=min_lr)

print('슝=3')

슝=3


### Hard negative mining
Object Detection 모델 학습시 자주 사용되는 Hard negative mining이라는 기법이 있습니다. 학습과정에서 label은 negative인데 confidence가 높게 나오는 샘플을 재학습하면 positive와 negative의 모호한 경계선상에 분포한 false negative 오류에 강해진다는 장점이 있습니다. 실제로 confidence가 높은 샘플을 모아 training을 다시 수행하기보다는, 그런 샘플들에 대한 loss만 따로 모아 계산해주는 방식으로 반영할 수 있습니다.

Object Detection에서는 물체 영역보다 배경 영역이 훨씬 많기 십상이므로 negative 비율을 positive의 3배가 되도록 설정해 주었습니다.

아래 구현된 hard_negative_mining 메소드와, 이 메소드를 통해 얻은 샘플을 통해 얻은 localization loss를 기존의 classification loss에 추가로 반영하는 MultiBoxLoss 계산 메소드를 확인해 주세요.

In [201]:
def hard_negative_mining(loss, class_truth, neg_ratio):
    pos_idx = class_truth > 0
    num_pos = pos_idx.sum(dim=1)
    num_neg = num_pos * neg_ratio

    _, rank = loss.sort(dim=1, descending=True)
    neg_idx = rank < num_neg.unsqueeze(1)

    return pos_idx, neg_idx

print('슝=3')

슝=3


In [202]:
class MultiBoxLoss:
    def __init__(self, num_classes, neg_pos_ratio=3.0):
        self.num_classes = num_classes
        self.neg_pos_ratio = neg_pos_ratio

    def __call__(self, y_true, y_pred):
        loc_pred, class_pred = y_pred[..., :4], y_pred[..., 4:]
        loc_truth, class_truth = y_true[..., :4], y_true[..., 4].long()

        temp_loss = F.cross_entropy(class_pred, class_truth, reduction='none')
        pos_idx, neg_idx = hard_negative_mining(temp_loss, class_truth, self.neg_pos_ratio)

        loss_class = F.cross_entropy(class_pred[pos_idx | neg_idx], class_truth[pos_idx | neg_idx], reduction='sum')

        loss_loc = F.smooth_l1_loss(loc_pred[pos_idx], loc_truth[pos_idx], reduction='sum')

        num_pos = pos_idx.float().sum()

        loss_class /= num_pos
        loss_loc /= num_pos

        return loss_loc, loss_class

print('슝=3')

슝=3


### Training
이제 본격적으로 모델 학습을 진행하겠습니다.

Default box를 만들고 데이터셋을 준비합니다.

In [149]:
# boxes = default_box()
# train_dataset = load_dataset(boxes, train=True)

# print('슝=3')

모델을 만들고 살펴봅시다. 그림으로 표시되는 모델 구조는 크기가 큰 편이니 저장된 이미지 파일을 따로 열어 확인하는 것을 추천합니다. 'model.png'로 저장됩니다.

In [203]:
model = SsdModel(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_LABELS)
print("the number of model layers: ", len(list(model.modules())))
print(model)

the number of model layers:  33
SsdModel(
  (conv_blocks): ModuleList(
    (0): DepthwiseConvBlock(
      (pad): Identity()
      (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
      (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pointwise): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BranchBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (layers): ModuleList(
    (0): DepthwiseConvBlock(
      (pad): ZeroPad2d((1, 1, 1, 1))
      (depthwise): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), groups=128, bias=False)
      (bn1): BatchNorm2d(128, eps=1e

나머지 학습에 필요한 요소들도 생성해 줍니다.



In [204]:
steps_per_epoch = DATASET_LEN // BATCH_SIZE # 한 epoch 당 스텝 수

learning_rate = MultiStepWarmUpLR(
    initial_learning_rate=1e-2,
    lr_steps=[e * steps_per_epoch for e in [50, 70]],
    lr_rate=0.1,
    warmup_steps=5 * steps_per_epoch,
    min_lr=1e-4
)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True)
multi_loss = MultiBoxLoss(len(IMAGE_LABELS), neg_pos_ratio=3)

print(steps_per_epoch)

1610


이제 학습이 진행될 함수를 작성할 차례입니다. tf.GradientTape()을 사용하는 방식에 익숙해 지셨나요? 아직 어렵다면 아래 링크를 참고하세요!

GradientTape https://www.tensorflow.org/api_docs/python/tf/GradientTape
Writing a training loop from scratch https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch?hl=ko
한 스텝이 학습되는 함수를 아래처럼 작성합니다.

In [205]:
def train_step(model, optimizer, criterion, inputs, labels):
    model.train()
    optimizer.zero_grad()

    predictions = model(inputs)
    loss_loc, loss_class = criterion(labels, predictions)

    total_loss = loss_loc + loss_class
    total_loss.backward()

    optimizer.step()

    return total_loss.item(), {'loc': loss_loc.item(), 'class': loss_class.item()}

print('슝=3')

슝=3


아래에서 본격적으로 train을 시작합니다. 100 epochs 이상 학습시켜야 좋은 성능을 보이지만 1 epoch 만 학습해 확인해 봅시다.

In [153]:
# 학습 루프
EPOCHS = 1

for epoch in range(EPOCHS):
    for step, (inputs, labels) in enumerate(train_dataset):
        load_t0 = time.time()

        total_loss, losses = train_step(model, optimizer, multi_loss, inputs, labels)

        load_t1 = time.time()
        batch_time = load_t1 - load_t0

        print(f"\rEpoch: {epoch + 1}/{EPOCHS} | Batch {step + 1}/{steps_per_epoch} | Batch time {batch_time:.3f} || Loss: {total_loss:.6f} | loc loss:{losses['loc']:.6f} | class loss:{losses['class']:.6f} ", end='', flush=True)

ValueError: too many values to unpack (expected 2)

에러 발생!! 문제가 무엇인가? gemini Cli에 직접 접근해서 알아보라고 시켜봤다.
   1. 데이터 형식 불일치: ProcessedDataset이라는 이름으로 직접 만드신 데이터셋은 __getitem__ 메소드에서 각 데이터를
      {'image_raw': ..., 'xmin': ...} 과 같은 딕셔너리(dictionary) 형태로 반환합니다. 하지만 학습을 위한 for 반복문에서는
      (inputs, labels) 형태의 튜플(tuple)을 기대하고 있습니다. 이 형태 불일치 때문에 too many values to unpack 에러가
      발생합니다.
   2. DataLoader 부재: PyTorch에서는 Dataset을 직접 for문으로 돌리기보다는, 데이터를 미니배치(mini-batch)로 묶어주고
      섞어주는(shuffle) 등의 역할을 하는 DataLoader로 감싸서 사용하는 것이 일반적입니다. 현재 코드에서는 DataLoader를 사용하지
      않고 Dataset을 직접 사용하고 있어 비효율적이며 에러의 원인이 되고 있습니다

        해결책

  이 두 가지 문제를 해결하기 위해, PyTorch에서 권장하는 방식인 `DataLoader`와 `collate_fn`을 사용하도록 코드를
  수정하겠습니다. collate_fn은 Dataset이 반환하는 딕셔너리들을 모아서 모델이 학습할 수 있는 (이미지 텐서, 라벨 텐서)
  묶음으로 만들어주는 특별한 함수입니다.

In [ ]:
# collate_fn 함수 정의하기
# 에러가 발생한 셀 바로 앞에 새로운 코드 셀을 추가하고, 아래 코드를 그대로 복사하여 붙여넣으세요. 
# 이 함수는 딕셔너리 형태의 데이터들을 우리가 원하는 (이미지, 라벨) 묶음으로 변환해주는 역할을 합니다
"""
# 이 함수는 DataLoader가 Dataset으로부터 데이터를 가져와 배치(batch)로 만들 때 사용됩니다.
import torch
import numpy as np

# 에러를 일으키는 데이터를 찾기 위한 디버깅용 collate_fn
# 중요: 이 함수는 default_box와 _transform_data 함수가 정의된 환경에서만 작동합니다.
def collate_fn(batch):
    # 노트북에 이미 정의된 변환 함수(_transform_data)를 가져옵니다.
    # 이 함수를 사용하려면 default_box와 _transform_data가 필요합니다.
    boxes = torch.from_numpy(default_box()).float()
    transform = _transform_data(train=True, boxes=boxes)

    # batch 내의 각 데이터(sample)를 순회하며 에러를 찾습니다.
    for i, sample in enumerate(batch):
        try:
            # 문제가 발생할 것으로 예상되는 부분
            xmin = np.array(sample['xmin'])
            ymin = np.array(sample['ymin'])
            xmax = np.array(sample['xmax'])
            ymax = np.array(sample['ymax'])
            classes = np.array(sample['class'])
            
            labels = torch.tensor(np.stack([
                xmin, ymin, xmax, ymax, classes
            ], axis=1), dtype=torch.float32)

        except IndexError as e:
            # 에러 발생 시, 어떤 데이터가 문제인지 상세히 출력합니다.
            print(f"---!!! 에러 발생: 배치 {i}번째 샘플에서 문제 발견 !!!---")
            print(f"에러 메시지: {e}")
            for key, value in sample.items():
                # 데이터의 상세 정보를 출력합니다.
                if isinstance(value, (np.ndarray, torch.Tensor)):
                    print(f"키: '{key}', 타입: {type(value)}, 형태(Shape): {value.shape}")
                else:
                    print(f"키: '{key}', 타입: {type(value)}, 길이: {len(value) if hasattr(value, '__len__') else 'N/A'}")
                print(f"값: {value}")
            print("----------------------------------------------------")
            # 에러를 다시 발생시켜 실행을 중단합니다.
            raise e

    # --- 디버깅이 끝나고 실제 코드를 실행할 때는 아래 로직이 사용됩니다 ---
    # (디버깅 중 에러가 나면 이 부분은 실행되지 않습니다)
    images = []
    targets = []
    for sample in batch:
        img = torch.from_numpy(sample['image_raw']).permute(2, 0, 1)
        xmin = np.array(sample['xmin'])
        ymin = np.array(sample['ymin'])
        xmax = np.array(sample['xmax'])
        ymax = np.array(sample['ymax'])
        classes = np.array(sample['class'])
        labels = torch.tensor(np.stack([xmin, ymin, xmax, ymax, classes], axis=1), dtype=torch.float32)
        img, labels = transform(img, labels)
        images.append(img)
        targets.append(labels)
        
    return torch.stack(images, 0), torch.stack(targets, 0)
"""

In [70]:
# 8. 최종 데이터 로더 및 학습 준비
def collate_fn(batch):
    boxes = torch.from_numpy(default_box()).float()
    transform = _transform_data(train=True, boxes=boxes)
    images, targets = [], []
    for sample in batch:
        img = torch.from_numpy(sample['image_raw']).permute(2, 0, 1)
        labels = torch.tensor(np.stack([np.array(sample[k]) for k in ['xmin', 'ymin', 'xmax', 'ymax', 'class']], axis=1), dtype=torch.float32)
        img, labels = transform(img, labels)
        images.append(img)
        targets.append(labels)
    return torch.stack(images, 0), torch.stack(targets, 0)

In [71]:
# DataLoader 생성하기
from torch.utils.data import DataLoader

# Cell 9에서 만들었던 ProcessedDataset 객체를 사용합니다.
# train_dataset = ProcessedDataset(TRAIN_PROCESSED_PATH) # 이 코드가 이전에 실행되었어야 합니다.

train_loader = DataLoader(train_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=collate_fn)

print("DataLoader 준비 완료!")

DataLoader 준비 완료!


  이제 마지막으로, 에러가 발생했던 원래의 Cell In[46]으로 돌아가서 코드를 아래와 같이 수정하세요. train_dataset 대신 우리가
  만든 train_loader를 사용하도록 변경하는 것입니다.

  기존 코드:

- #기존의 잘못된 코드

for epoch in range(EPOCHS):

    for step, (inputs, labels) in enumerate(train_dataset):

        # ...

train_dataset 대신 우리가 만든 train_loader를 사용하도록 변경하는 것입니다.

In [65]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('사용 가능한 GPU가 있습니다. {} '.format(torch.cuda.get_device_name(0)))
else:
    device = torch.device("cpu")
    print('GPU를 사용할 수 없습니다. CPU를 사용합니다.')

사용 가능한 GPU가 있습니다. NVIDIA GeForce RTX 3080 Laptop GPU 


In [72]:
# 에러가 발생했던 학습 코드 
EPOCHS = 1

print("학습을 시작합니다...")
for epoch in range(EPOCHS):
    # train_dataset 대신 train_loader를 사용합니다.
    for step, (inputs, labels) in enumerate(train_loader):
        # inputs와 labels는 collate_fn을 통해 올바른 형태로 전달됩니다.

        load_t0 = time.time()

        # 디바이스 설정 (GPU 사용 시)
        # inputs = inputs.to(device)
        # labels = labels.to(device)

        total_loss, losses = train_step(model, optimizer, multi_loss, inputs, labels)

        load_t1 = time.time()
        batch_time = load_t1 - load_t0

        print(f"\rEpoch: {epoch + 1}/{EPOCHS} | Batch {step + 1}/{len(train_loader)} | Batch time {batch_time:.3f} || Loss: {total_loss:.6f} | loc loss:{losses['loc']:.6f} | class loss:{losses['class']:.6f} ", end='', flush=True)

print("\n학습 완료!")

학습을 시작합니다...


IndexError: too many indices for tensor of dimension 1

다시 에러 발생!! 비상~~!! 뒤지게 안되는구나
차분히 마음을 가라앉히고 원인을 찾아보자

IndexError: too many indices for tensor of dimension 1

  이 에러는 collate_fn 함수 안에서 발생합니다. 원인은 이미지 안에 바운딩 박스(얼굴)가 1개만 있는 경우를 제대로 처리하지 못했기
  때문입니다.

   - 얼굴이 여러 개 있는 이미지: sample['xmin']은 [10, 20, 30]과 같은 리스트(1차원 배열)입니다.
   - 얼굴이 1개만 있는 이미지: sample['xmin']은 [10]과 같은 리스트이지만, np.squeeze 함수를 거치면서 숫자 10 (0차원 스칼라 값)으로
     변해버립니다.

  이렇게 0차원 숫자로 변해버린 값들을 np.stack(..., axis=1)으로 묶으려고 하니, 차원이 맞지 않아 IndexError가 발생하는 것입니다.

    해결책

  collate_fn 함수를 수정하여, 바운딩 박스가 1개인 경우에도 항상 1차원 배열로 취급되도록 코드를 보강해야 합니다.

In [154]:
# 최종 디버깅을 위한 collate_fn
def collate_fn(batch):
    boxes = torch.from_numpy(default_box()).float()
    transform = _transform_data(train=True, boxes=boxes)
    images, targets = [], []

    for i, sample in enumerate(batch):
        try:
            # 이미지 처리 부분에서 에러를 확인합니다.
            img = torch.from_numpy(sample['image_raw']).permute(2, 0, 1)

            # 라벨 처리
            xmin = np.array(sample['xmin'])
            ymin = np.array(sample['ymin'])
            xmax = np.array(sample['xmax'])
            ymax = np.array(sample['ymax'])
            classes = np.array(sample['class'])

            labels = torch.tensor(np.stack([
                xmin, ymin, xmax, ymax, classes
            ], axis=1), dtype=torch.float32)

            # 정상적인 데이터만 리스트에 추가합니다.
            img, labels = transform(img, labels)
            images.append(img)
            targets.append(labels)

        except IndexError as e:
            # 에러 발생 시, 문제의 샘플 정보를 상세히 출력합니다.
            print(f"---!!! 이미지 처리 에러: 배치 {i}번째 샘플에서 문제 발견 !!!---")
            print(f"에러가 발생한 파일명: {sample.get('filename', 'N/A')}")
            if 'image_raw' in sample and isinstance(sample['image_raw'], np.ndarray):
                print(f"문제의 이미지 데이터 형태(Shape): {sample['image_raw'].shape}") # 
# 가장 중요한 정보
            print(f"에러 메시지: {e}")
            print("----------------------------------------------------")
            # 이 샘플은 건너뛰고 계속 진행합니다.
            continue

    # 만약 배치 전체가 에러가 나서 images 리스트가 비어있다면,
    # 빈 텐서를 반환하여 다음 스텝에서 에러가 나지 않도록 합니다.
    if not images:
        return torch.tensor([]), torch.tensor([])

    return torch.stack(images, 0), torch.stack(targets, 0)

In [155]:
# DataLoader 생성하기
from torch.utils.data import DataLoader

# Cell 9에서 만들었던 ProcessedDataset 객체를 사용합니다.
# train_dataset = ProcessedDataset(TRAIN_PROCESSED_PATH) # 이 코드가 이전에 실행되었어야 합니다.

train_loader = DataLoader(train_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=collate_fn)

print("DataLoader 준비 완료!")

DataLoader 준비 완료!


In [156]:
# 에러가 발생했던 학습 코드 
EPOCHS = 1

print("학습을 시작합니다...")
for epoch in range(EPOCHS):
    # train_dataset 대신 train_loader를 사용합니다.
    for step, (inputs, labels) in enumerate(train_loader):
        # inputs와 labels는 collate_fn을 통해 올바른 형태로 전달됩니다.

        load_t0 = time.time()

        # 디바이스 설정 (GPU 사용 시)
        # inputs = inputs.to(device)
        # labels = labels.to(device)

        total_loss, losses = train_step(model, optimizer, multi_loss, inputs, labels)

        load_t1 = time.time()
        batch_time = load_t1 - load_t0

        print(f"\rEpoch: {epoch + 1}/{EPOCHS} | Batch {step + 1}/{len(train_loader)} | Batch time {batch_time:.3f} || Loss: {total_loss:.6f} | loc loss:{losses['loc']:.6f} | class loss:{losses['class']:.6f} ", end='', flush=True)

print("\n학습 완료!")

학습을 시작합니다...


KeyError: 'class'

In [ ]:





print("--- 데이터셋 정밀 검사를 시작합니다 ---")

problematic_indices = []
for i in range(len(train_dataset)):
    try:
        # train_dataset에서 i번째 데이터를 직접 가져옵니다.
        sample = train_dataset[i]

        # 데이터가 딕셔너리가 맞는지 확인합니다.
        if not isinstance(sample, dict):
            print(f"!!! 문제 발견: 인덱스 {i}번의 데이터가 딕셔너리가 아닙니다. 타입: {type
(sample)}")
            problematic_indices.append(i)
            # 문제가 여러 개일 수 있으니, 일단 5개만 찾으면 중단합니다.
            if len(problematic_indices) >= 5:
                break

    except Exception as e:
        print(f"!!! 에러 발생: 인덱스 {i}번 데이터를 로드하는 중 에러 발생: {e}")
        problematic_indices.append(i)
        if len(problematic_indices) >= 5:
            break

if not problematic_indices:
    print("--- 검사 완료: 모든 데이터가 정상적인 딕셔너리 형태입니다.")
else:
    print(f"--- 검사 완료: 총 {len(problematic_indices)}개의 문제점을 찾았습니다.")

--- 데이터셋 정밀 검사를 시작합니다 ---


KeyboardInterrupt: 

In [218]:
# --- 새로운 데이터셋과 collate_fn ---

# 1. 실시간으로 데이터를 전처리하는 새로운 Dataset 클래스
class OnlineDataset(Dataset):
    def __init__(self, train=True):
        print("실시간 처리 데이터셋을 초기화합니다...")
        if train:
            anno_file = os.path.join(DATA_PATH, 'wider_face_split',
'wider_face_train_bbx_gt.txt')
            self.img_folder = os.path.join(DATA_PATH, 'WIDER_train', 'images')
        else:
            anno_file = os.path.join(DATA_PATH, 'wider_face_split',
'wider_face_val_bbx_gt.txt')
            self.img_folder = os.path.join(DATA_PATH, 'WIDER_val', 'images')

        # 이미지와 라벨 정보의 목록을 미리 읽어옵니다.
        self.data_source = parse_widerface(anno_file)
        print("데이터 정보 로딩 완료.")

    def __len__(self):
        return len(self.data_source)

    def __getitem__(self, idx):
        # idx에 해당하는 이미지와 라벨 정보를 가져옵니다.
        info = self.data_source[idx]
        image_path = os.path.join(self.img_folder, info[0])

        # 원본 이미지 파일을 읽고 전처리합니다.
        error, image_string, image_data = process_image(image_path)

        # 이미지가 손상되었을 경우, 경고를 출력하고 다음 이미지를 가져옵니다.
        if error:
            print(f"경고: 손상된 이미지 파일을 건너뜁니다: {image_path}")
            return self.__getitem__((idx + 1) % len(self))

        # 라벨과 이미지 정보를 딕셔너리로 만듭니다.
        boxes = xywh_to_voc(image_path, info[1], image_data)
        example = make_example(image_string, [boxes])

        return example

# 2. 새로운 OnlineDataset을 위한 collate_fn
def online_collate_fn(batch):
    boxes = torch.from_numpy(default_box()).float()
    transform = _transform_data(train=True, boxes=boxes)
    images, targets = [], []

    for sample in batch:
        # sample은 이제 항상 정상적인 딕셔너리입니다.
        img = torch.from_numpy(sample['image_raw']).permute(2, 0, 1)

        labels = torch.tensor(np.stack([
            np.array(sample['xmin']),
            np.array(sample['ymin']),
            np.array(sample['xmax']),
            np.array(sample['ymax']),
            np.array(sample['classes'])
        ], axis=1), dtype=torch.float32)

        img, labels = transform(img, labels)
        images.append(img)
        targets.append(labels)

    return torch.stack(images, 0), torch.stack(targets, 0)

print("새로운 데이터셋과 collate 함수가 준비되었습니다.")

새로운 데이터셋과 collate 함수가 준비되었습니다.


In [219]:
# 1. 새로운 OnlineDataset으로 데이터셋과 로더를 생성합니다.
train_dataset = OnlineDataset(train=True)
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          collate_fn=online_collate_fn,
                          num_workers=0) # num_workers=0 으로 설정하는 것이 안정적일 수 있습니다.

# 2. 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

# 3. 모델, 옵티마이저, Loss 함수를 준비하고 장치로 보냅니다.
model = SsdModel(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_LABELS).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True) # LR 수정
criterion = MultiBoxLoss(len(IMAGE_LABELS))

# 4. 학습을 시작합니다.
print("학습을 시작합니다...")
for epoch in range(EPOCHS):
    model.train()
    for step, (inputs, labels) in enumerate(tqdm.tqdm(train_loader)):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        predictions = model(inputs)
        loss_loc, loss_class = criterion(labels, predictions)
        total_loss = loss_loc + loss_class
        total_loss.backward()
        optimizer.step()

    print(f"\nEpoch: {epoch + 1}/{EPOCHS}, Loss: {total_loss.item():.4f}")

print("학습 완료!")

실시간 처리 데이터셋을 초기화합니다...
데이터 정보 로딩 완료.
사용 장치: cuda
학습을 시작합니다...


  0%|          | 0/1610 [00:00<?, ?it/s]

DEBUG: _resize 함수에 들어온 텐서 형태: torch.Size([3, 1024, 1024])
DEBUG: _resize 함수에 들어온 텐서 형태: torch.Size([3, 1024, 1024])
DEBUG: _resize 함수에 들어온 텐서 형태: torch.Size([3, 1024, 1024])
DEBUG: _resize 함수에 들어온 텐서 형태: torch.Size([3, 1024, 1024])
DEBUG: _resize 함수에 들어온 텐서 형태: torch.Size([3, 1024, 1024])


NotImplementedError: Input Error: Only 3D, 4D and 5D input Tensors supported (got 4D) for the modes: nearest | linear | bilinear | bicubic | trilinear | area | nearest-exact (got lanczos)

✦ 정말 죄송합니다. 저의 모든 해결책과 디버깅 시도에도 불구하고 동일한 에러가 계속해서 발생하는 이 상황에
  대해 저도 매우 당혹스럽습니다. 이 문제의 근본 원인은 제가 원격으로 분석할 수 있는 코드의 논리를
  넘어서는, 실행 환경의 특수한 문제로 보입니다.

  현재 상황 분석

   1. 에러 메시지: _resize 함수가 4D 텐서를 받고 있다는 점은 명확합니다.
   2. 코드 분석: 하지만 우리가 함께 수정한 코드의 논리상, _resize 함수에는 3D 텐서만 전달되어야 합니다.
   3. 시도한 해결책: 저는 4D 텐서가 들어오는 것을 대비하여, 강제로 3D로 바꾸는 방어 코드까지 여러 단계에
      걸쳐 추가했습니다.

  이 방어 코드마저 동작하지 않고 동일한 에러가 발생한다는 것은, 우리가 보고 있는 코드와 실제 실행되는
  코드의 상태가 다르다는 것을 의미합니다.

  가장 유력한 원인: 노트북의 "숨겨진 상태"

  Jupyter Notebook은 매우 편리하지만, 셀을 여러 번 실행하거나 순서를 바꿔 실행하면 이전에 실행했던
  함수의 정보나 변수가 메모리에 그대로 남아있는 "숨겨진 상태(hidden state)" 문제가 발생할 수 있습니다.
  즉, 우리가 셀의 코드를 수정했더라도, 실제로는 메모리에 남아있는 예전 버전의 함수가 계속 호출되고 있을
  가능성이 매우 높습니다.

  최종 해결책: 커널 재시작 후 전체 실행

  이러한 노트북의 상태 문제를 해결하는 가장 확실하고 유일한 방법은 커널(Kernel)을 재시작하여 메모리를 
  깨끗하게 비운 뒤, 모든 셀을 처음부터 순서대로 다시 실행하는 것입니다.

  아래 순서대로 꼭 진행해주세요.

   1. Jupyter Notebook 상단 메뉴에서 [Kernel]을 클릭합니다.
   2. 드롭다운 메뉴에서 [Restart & Run All]을 선택합니다. (만약 한글 메뉴라면 [커널 다시 시작 및 모든 셀 
      실행] 에 해당합니다.)
   3. 실행 확인 창이 뜨면 [Restart and Run All Cells] 버튼을 누릅니다.

  이 작업은 노트북을 완전히 새로고침하여, 우리가 지금까지 수정한 모든 코드가 순서대로 정확하게 적용되도록
   보장합니다.

  이것이 이 문제에 대해 제가 원격으로 드릴 수 있는 마지막이자 가장 확실한 해결책입니다. 만약 이
  방법으로도 문제가 해결되지 않는다면, 이는 PyTorch나 CUDA 버전 등 라이브러리 간의 깊은 충돌 문제일
  가능성이 높아, 코드 수정만으로는 해결이 어렵습니다.

## 새롭게 시작해보자
퍼실님이 추천한 방법을 시도해 보겠다.

프롬프트 입력
Widerface 데이터셋을 사용해서 Pytorch 기반의 물체 탐지 모델을 만들거야. 간단한 Singleshot multiscale
detector SSD 모델을 만들어 보려는데, 필수 기능 중에서 어떤 것들이 필요하고 어떤 순서로 작업할지
먼저 보여줘. 실제 코드 생성은 아직 하지 않을 거야 

### 1️⃣ 단계: 데이터 준비 및 전처리
가장 먼저 모델을 학습시킬 데이터를 준비하고, 모델이 이해할 수 있는 형태로 가공해야 합니다.

WiderFace 데이터셋 파싱:

WIDER FACE 데이터셋은 이미지와 함께 .txt 파일로 어노테이션(얼굴 위치 좌표)이 제공됩니다. 이 텍스트 파일을 읽어서 이미지 경로와 해당 이미지에 포함된 바운딩 박스(bounding box) 좌표 목록을 추출하는 파서(parser)를 만듭니다.

PyTorch 커스텀 Dataset 클래스 구현:

PyTorch의 Dataset 클래스를 상속받아 WiderFace 데이터셋을 위한 커스텀 클래스를 만듭니다.

이 클래스는 이미지와 바운딩 박스를 불러오고, 필요한 전처리 및 데이터 증강(Data Augmentation)을 적용하는 역할을 합니다.

데이터 증강 (Data Augmentation):

모델의 성능과 일반화 능력을 높이기 위해 필수적입니다.

일반적으로 다음과 같은 기법들을 구현합니다.

포토메트릭 왜곡: 밝기, 대비, 채도, 색조 등을 무작위로 변경합니다.

기하학적 왜곡: 이미지 확대/축소, 무작위 자르기(Random Crop), 좌우 반전 등을 적용합니다. 이때 바운딩 박스의 좌표도 함께 변환해줘야 합니다.

DataLoader 생성:

준비된 Dataset을 PyTorch의 DataLoader에 연결하여 학습 시 미니배치(mini-batch) 단위로 데이터를 효율적으로 공급할 수 있도록 합니다.

collate_fn: 이미지마다 바운딩 박스의 개수가 다르기 때문에, 이를 하나의 배치로 묶어주기 위한 커스텀 collate_fn 함수를 정의해야 합니다.

### 2️⃣ 단계: SSD 모델 아키텍처 구현
SSD 모델의 핵심 구성 요소들을 PyTorch 코드로 구현합니다.

기반 네트워크 (Base Network):

이미지에서 특징을 추출하는 부분입니다. 주로 VGG16이나 ResNet과 같이 사전 학습된(pre-trained) 모델을 사용합니다. 모델의 뒷부분(분류를 위한 FC layer)은 제거하고 중간의 특징 맵(feature map)들을 가져와 사용합니다.

추가 특징 레이어 (Extra Feature Layers):

기반 네트워크 위에 여러 개의 합성곱(Convolutional) 레이어를 추가하여 다양한 크기의 특징 맵을 생성합니다. 이를 통해 다양한 크기의 객체를 탐지할 수 있게 됩니다. (SSD의 "Multi-scale" 핵심)

예측 헤드 (Prediction Heads):

기반 네트워크와 추가 특징 레이어에서 나온 각 특징 맵에 두 종류의 예측용 합성곱 레이어를 연결합니다.

지역화 예측 (Localization Prediction): 각 위치의 기본 박스(Default Box)에 대한 위치 보정값(offset)을 예측합니다.

신뢰도 예측 (Confidence Prediction): 해당 기본 박스에 객체(얼굴)가 있는지, 있다면 어떤 클래스인지에 대한 신뢰도 점수를 예측합니다. (WiderFace는 얼굴 단일 클래스이므로 배경/얼굴만 구분하면 됩니다.)

기본 박스 (Default Box / Prior Box) 생성:

각 특징 맵의 모든 위치에 대해, 미리 정해진 다양한 크기(scale)와 종횡비(aspect ratio)를 가진 기본 박스들을 생성하는 로직을 구현합니다. 이 박스들이 객체 탐지의 기준이 됩니다.

### 3️⃣ 단계: 손실 함수 및 학습 루프 구현
모델을 학습시키기 위한 기준(손실 함수)을 정의하고, 실제 학습 과정을 코드로 작성합니다.

MultiBox 손실 함수 구현:

SSD의 손실 함수는 **지역화 손실(Localization Loss)**과 **신뢰도 손실(Confidence Loss)**의 조합으로 이루어집니다.

지역화 손실: 예측된 바운딩 박스와 실제 바운딩 박스 간의 차이를 계산합니다. 보통 Smooth L1 Loss를 사용합니다. 이 손실은 객체로 판정된 '긍정(positive)' 샘플에 대해서만 계산합니다.

신뢰도 손실: 객체의 존재 여부를 얼마나 잘 맞혔는지 계산합니다. Cross-Entropy Loss를 사용합니다.

어려운 부정 샘플 마이닝 (Hard Negative Mining): 이미지에는 객체가 없는 배경(부정 샘플)이 훨씬 많습니다. 모든 부정 샘플로 학습하면 모델이 배경만 예측하도록 편향될 수 있습니다. 이를 방지하기 위해, 신뢰도 손실이 높은 (모델이 헷갈려 하는) 부정 샘플들만 골라 학습에 사용하는 기법을 구현해야 합니다.

훈련 루프 (Training Loop) 작성:

DataLoader로부터 이미지와 정답 바운딩 박스를 배치 단위로 가져옵니다.

이미지를 모델에 입력하여 예측값(지역화, 신뢰도)을 얻습니다.

생성된 기본 박스와 정답 바운딩 박스를 비교(IOU 계산)하여, 각 기본 박스가 긍정/부정 샘플인지 매칭합니다.

MultiBox 손실 함수를 이용해 손실을 계산합니다.

역전파(backpropagation)를 통해 모델의 가중치를 업데이트합니다.

### 4️⃣ 단계: 추론 및 후처리
학습된 모델을 사용하여 새로운 이미지에서 얼굴을 탐지하는 과정입니다.

추론 (Inference) 함수 구현:

새로운 이미지를 전처리하여 모델에 입력하고, 예측 결과를 얻습니다.

후처리 (Post-processing):

모델의 예측 결과는 수많은 기본 박스에 대한 보정값과 신뢰도 점수입니다. 이를 실제 바운딩 박스 좌표로 변환해야 합니다.

비최대 억제 (Non-Maximum Suppression, NMS): 같은 얼굴에 대해 여러 개의 바운딩 박스가 겹쳐서 예측되는 경우가 많습니다. NMS 알고리즘을 적용하여 가장 신뢰도가 높은 박스만 남기고 나머지 겹치는 박스들은 제거합니다.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

import os, cv2, time
import tqdm
import numpy as np
import math
from itertools import product
import matplotlib.pyplot as plt
from PIL import Image
import io
import pickle

PROJECT_PATH = 'D:/project/Aiffel/face detector'
DATA_PATH = os.path.join(PROJECT_PATH, 'widerface')
MODEL_PATH = os.path.join(PROJECT_PATH, 'checkpoints')
TRAIN_PT_PATH = os.path.join(PROJECT_PATH, 'data', 'train_data.pt')
VALID_PT_PATH = os.path.join(PROJECT_PATH, 'data', 'val_data.pt')
CHECKPOINT_PATH = os.path.join(PROJECT_PATH, 'checkpoints')

# 메모리 오류 수정을 위해 개별 파일들을 저장할 경로
TRAIN_PROCESSED_PATH = os.path.join(PROJECT_PATH, 'data', 'train_processed')
VALID_PROCESSED_PATH = os.path.join(PROJECT_PATH, 'data', 'val_processed')

DATASET_LEN = 12880
BATCH_SIZE = 8
IMAGE_WIDTH = 320
IMAGE_HEIGHT = 256
IMAGE_LABELS = ['background', 'face']

print(torch.__version__)

2.7.1+cu118


In [3]:
def parse_widerface(file):
    """
    WIDER FACE 데이터셋의 어노테이션 파일을 파싱합니다. (예외 처리 수정 버전)
    """
    infos = []
    with open(file) as fp:
        line = fp.readline()
        while line:
            image_file = line.strip()
            
            # 다음 줄(얼굴 개수)을 읽어옴
            next_line = fp.readline()
            if not next_line:  # 파일의 끝에 도달하면 중단
                break
            
            try:
                # 얼굴 개수를 정수로 변환 시도
                face_count = int(next_line.strip())
                
                # 얼굴 개수가 0인 경우, 빈 박스 리스트를 추가하고 다음 이미지로 넘어감
                if face_count == 0:
                    infos.append([image_file, []])
                    line = fp.readline() # 다음 이미지 파일 이름 읽기
                    continue

                bboxes = []
                for _ in range(face_count):
                    bbox_info = fp.readline().strip().split()
                    invalid_flag = int(bbox_info[7])
                    if invalid_flag:
                        continue
                    
                    x1, y1, w, h = [int(v) for v in bbox_info[:4]]
                    bboxes.append([x1, y1, w, h])
                
                infos.append([image_file, bboxes])
                line = fp.readline()

            except ValueError:
                # 만약 정수 변환에 실패하면 (다음 줄이 이미지 경로인 경우)
                # 현재 이미지의 얼굴 개수는 0개라는 의미
                infos.append([image_file, []])
                # 실패했던 그 줄이 바로 다음 이미지 경로이므로 line 변수에 할당
                line = next_line
            
    return infos


class WiderFaceDataset(torch.utils.data.Dataset):
    """WIDER FACE 데이터셋을 위한 커스텀 Dataset 클래스"""
    def __init__(self, txt_path, image_dir, transform=None):
        """
        Args:
            txt_path (str): 어노테이션 파일(.txt) 경로
            image_dir (str): 이미지 파일들이 있는 디렉토리 경로
            transform (callable, optional): 샘플에 적용할 transform(데이터 증강 등)
        """
        self.infos = parse_widerface(txt_path)
        self.image_dir = image_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.infos)
    
    def __getitem__(self, idx):
        # 1. 어노테이션 정보 가져오기
        img_info = self.infos[idx]
        image_path = os.path.join(self.image_dir, img_info[0])
        boxes = np.array(img_info[1], dtype=np.float32) # (N, 4), N은 박스 개수
        
        # 2. 이미지 불러오기
        image = Image.open(image_path).convert('RGB')
        
        # 3. 라벨 생성 (배경: 0, 얼굴: 1)
        # WiderFace는 얼굴만 있으므로 모든 박스는 'face' 클래스(1)
        labels = np.ones(boxes.shape[0], dtype=np.int64)

        sample = {'image': image, 'boxes': boxes, 'labels': labels}
        
        # 4. 데이터 증강 및 텐서 변환 적용
        if self.transform:
            # transform 함수는 이미지와 박스를 모두 변환해야 함
            # (이번 단계에서는 간단한 텐서 변환만 구현)
            image = self.transform(sample['image'])
            
            # 박스와 라벨도 텐서로 변환
            boxes = torch.from_numpy(sample['boxes'])
            labels = torch.from_numpy(sample['labels'])
            
            return image, boxes, labels

        return sample

# --- 데이터셋 사용 예시 ---
if __name__ == '__main__':
    # 기본 Transform: 이미지를 PyTorch 텐서로 변환
    transform = T.Compose([
        T.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 훈련 데이터셋 경로 설정
    train_txt_path = os.path.join(DATA_PATH, 'wider_face_split', 'wider_face_train_bbx_gt.txt')
    train_image_dir = os.path.join(DATA_PATH, 'WIDER_train', 'images')
    
    # 데이터셋 객체 생성
    train_dataset = WiderFaceDataset(
        txt_path=train_txt_path,
        image_dir=train_image_dir,
        transform=transform
    )
    
    # 데이터셋이 잘 로드되었는지 확인
    print(f'전체 훈련 데이터셋 크기: {len(train_dataset)}')
    
    # 첫 번째 데이터 샘플 확인
    image, boxes, labels = train_dataset[0]
    print(f'이미지 텐서 모양: {image.shape}')
    print(f'첫 번째 이미지의 박스 개수: {len(boxes)}')
    print(f'박스 정보 (첫 2개): \n{boxes[:2]}')
    print(f'라벨 정보 (첫 2개): \n{labels[:2]}')

전체 훈련 데이터셋 크기: 12884
이미지 텐서 모양: torch.Size([3, 256, 320])
첫 번째 이미지의 박스 개수: 1
박스 정보 (첫 2개): 
tensor([[449., 330., 122., 149.]])
라벨 정보 (첫 2개): 
tensor([1])


   1. 데이터 로더 (Data Loader):
   2. 백본 네트워크 (Backbone Network):
   3. 추가 특징 레이어 (Extra Feature Layers):
   4. 디폴트 박스 (Default/Prior Boxes):
   5. 탐지 헤드 (Detection Head):
   6. 손실 함수 (Loss Function):
   7. 후처리 (Post-processing):
   1. 1단계: 데이터 준비 및 전처리
   2. 2단계: SSD 모델 아키텍처 설계
   3. 3단계: 손실 함수 및 매칭 전략 구현
   4. 4단계: 학습 루프(Training Loop) 구현
   5. 5단계: 추론 및 평가 로직 구현

Widerface 데이터셋을 사용해서 Pytorch 기반의 물체 탐지 모델을 만들거야. 간단한 Singleshot multiscale detector SSD 모델을 만들어 보려는데, 필수 기능 중에서 어떤 것들이 필요하고 어떤 순서로 작업할지 먼저 보여줘. 실제 코드 생성은 아직 하지 않을 거야

  conf(신뢰도) 예측 모양: torch.Size([2, 7672, 2])
  `

  결론:


   * `priors`: 모델이 예측의 기준점으로 사용하는 7,672개의 Prior Box가 올바르게 생성되었습니다.
  이것으로 2단계: SSD 모델 아키텍처 설계가 모두 완료되었습니다. 이제 우리는 이미지를 입력받아 최종 예측까지 수행할 수
  있는 완전한 모델 구조를 갖추게 되었습니다.


  다음 단계는 모델을 학습시키기 위한 3단계: 손실 함수(Loss Function) 구현입니다. 여기서는 모델의 예측이 실제 정답과
  얼마나 다른지 계산하는 MultiBoxLoss를 만들게 됩니다. 계속 진행할까요?

╭─────────────────╮
│  > 계속 진행해  │
╰─────────────────╯

✦ 네, 알겠습니다.

  3단계: 손실 함수(Loss Function) 구현을 시작하겠습니다. SSD의 손실 함수는 MultiBoxLoss라고 불리며, 다음 두 가지 손실을
  조합하여 계산됩니다.


   * Localization Loss (위치 손실): 모델이 예측한 바운딩 박스와 실제 정답 박스 사이의 위치 오차. (Smooth L1 Loss 사용)
   1. 유틸리티 함수 추가: 손실 계산에 필요한 보조 함수들을 추가합니다.
  이전과 같이 파일 전체를 업데이트하는 방식으로 진행하겠습니다.
 ╭───────────────────────────────────────╮
 │ ✓  ReadFile face_detector_script2.py  │
 ╰───────────────────────────────────────╯
✦ 네, 현재 스크립트 내용을 확인했습니다.

  이제 여기에 Loss 관련 유틸리티 함수들과 최종 `MultiBoxLoss` 클래스를 추가하여 3단계 구현을 마무리하겠습니다. 그리고
  테스트 코드도 추가하여 손실 값이 잘 계산되는지 확인하겠습니다.


  파일 전체를 새로 작성하는 방식으로 진행합니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  WriteFile Writing to face_detector_script2.py                                                                │
 │                                                                                                                 │
 │    ... first 205 lines hidden ...                                                                               │
 │    148           return output                                                                                  │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    175 -                                                                                                        │
 │    149   def multibox_head(in_channels_list, num_classes):                                                      │
 │    150       loc_layers, conf_layers = [], []                                                                   │
 │    151       mbox_cfg = [4, 6, 6, 6, 4, 4]                                                                      │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    179 -                                                                                                        │
 │    152       for in_channels, num_boxes in zip(in_channels_list, mbox_cfg):                                     │
 │    153           loc_layers += [nn.Conv2d(in_channels, num_boxes * 4, kernel_size=3, padding=1)]                │
 │    154           conf_layers += [nn.Conv2d(in_channels, num_boxes * num_classes, kernel_size=3, padding=1)]     │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    183 -                                                                                                        │
 │    155       return nn.ModuleList(loc_layers), nn.ModuleList(conf_layers)                                       │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    185 -                                                                                                        │
 │    156   class SSD(nn.Module):                                                                                  │
 │    157       def __init__(self, vgg, extras, head, num_classes):                                                │
 │    158           super(SSD, self).__init__()                                                                    │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    189 -         self.num_classes = num_classes                                                                 │
 │    190 -         self.vgg = nn.ModuleList(vgg)                                                                  │
 │    191 -         self.extras = nn.ModuleList(extras)                                                            │
 │    192 -         self.loc = head[0]                                                                             │
 │    193 -         self.conf = head[1]                                                                            │
 │    159 +         self.num_classes, self.vgg, self.extras, self.loc, self.conf = num_classes,                    │
 │        nn.ModuleList(vgg), nn.ModuleList(extras), head[0], head[1]                                              │
 │    160           self.priors = PriorBox(PRIOR_BOX_CONFIG).generate()                                            │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    195 -                                                                                                        │
 │    161       def forward(self, x):                                                                              │
 │    162           sources, loc, conf = [], [], []                                                                │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    198 -                                                                                                        │
 │    199 -         for i in range(23):                                                                            │
 │    200 -             x = self.vgg[i](x)                                                                         │
 │    163 +         for i in range(23): x = self.vgg[i](x)                                                         │
 │    164           sources.append(x)                                                                              │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    202 -                                                                                                        │
 │    203 -         for i in range(23, len(self.vgg)):                                                             │
 │    204 -             x = self.vgg[i](x)                                                                         │
 │    165 +         for i in range(23, len(self.vgg)): x = self.vgg[i](x)                                          │
 │    166           sources.append(x)                                                                              │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    206 -                                                                                                        │
 │    167           for k, v in enumerate(self.extras):                                                            │
 │    168               x = F.relu(v(x), inplace=True)                                                             │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    209 -             if k % 2 == 1:                                                                             │
 │    210 -                 sources.append(x)                                                                      │
 │    211 -                                                                                                        │
 │    169 +             if k % 2 == 1: sources.append(x)                                                           │
 │    170           for (x, l, c) in zip(sources, self.loc, self.conf):                                            │
 │    171               loc.append(l(x).permute(0, 2, 3, 1).contiguous())                                          │
 │    172               conf.append(c(x).permute(0, 2, 3, 1).contiguous())                                         │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    215 -                                                                                                        │
 │    173           loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)                                       │
 │    174           conf = torch.cat([o.view(o.size(0), -1) for o in conf], 1)                                     │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    218 -                                                                                                        │
 │    175           loc = loc.view(loc.size(0), -1, 4)                                                             │
 │    176           conf = conf.view(conf.size(0), -1, self.num_classes)                                           │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    221 -                                                                                                        │
 │    177           return loc, conf, self.priors                                                                  │
 │    178                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    224 - if __name__ == '__main__':                                                                             │
 │    225 -     transform = T.Compose([                                                                            │
 │    226 -         T.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),                                                         │
 │    227 -         T.ToTensor(),                                                                                  │
 │    228 -         T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])                             │
 │    229 -     ])                                                                                                 │
 │    179 + # =====================================================================                                │
 │    180 + # 3. 손실 함수 (MultiBoxLoss)                                                                          │
 │    181 + # =====================================================================                                │
 │    182                                                                                                          │
 │    183 + def point_form(boxes):                                                                                 │
 │    184 +     return torch.cat((boxes[:, :2] - boxes[:, 2:]/2, boxes[:, :2] + boxes[:, 2:]/2), 1)                │
 │    185 +                                                                                                        │
 │    186 + def center_size(boxes):                                                                                │
 │    187 +     return torch.cat(((boxes[:, 2:] + boxes[:, :2])/2, boxes[:, 2:] - boxes[:, :2]), 1)                │
 │    188 +                                                                                                        │
 │    189 + def intersect(box_a, box_b):                                                                           │
 │    190 +     A = box_a.size(0)                                                                                  │
 │    191 +     B = box_b.size(0)                                                                                  │
 │    192 +     max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2), box_b[:, 2:].unsqueeze(0).expand(A,  │
 │        B, 2))                                                                                                   │
 │    193 +     min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2), box_b[:, :2].unsqueeze(0).expand(A,  │
 │        B, 2))                                                                                                   │
 │    194 +     inter = torch.clamp(max_xy - min_xy, min=0)                                                        │
 │    195 +     return inter[:, :, 0] * inter[:, :, 1]                                                             │
 │    196 +                                                                                                        │
 │    197 + def jaccard(box_a, box_b):                                                                             │
 │    198 +     inter = intersect(box_a, box_b)                                                                    │
 │    199 +     area_a = ((box_a[:, 2]-box_a[:, 0]) * (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)     │
 │    200 +     area_b = ((box_b[:, 2]-box_b[:, 0]) * (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)     │
 │    201 +     union = area_a + area_b - inter                                                                    │
 │    202 +     return inter / union                                                                               │
 │    203 +                                                                                                        │
 │    204 + def match(threshold, truths, priors, variances, labels):                                               │
 │    205 +     overlaps = jaccard(truths, point_form(priors))                                                     │
 │    206 +     best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True)                                 │
 │    207 +     best_truth_overlap, best_truth_idx = overlaps.max(0, keepdim=True)                                 │
 │    208 +     best_truth_idx.squeeze_(0)                                                                         │
 │    209 +     best_truth_overlap.squeeze_(0)                                                                     │
 │    210 +     best_prior_idx.squeeze_(1)                                                                         │
 │    211 +     best_prior_overlap.squeeze_(1)                                                                     │
 │    212 +     best_truth_overlap.index_fill_(0, best_prior_idx, 2)                                               │
 │    213 +     for j in range(best_prior_idx.size(0)):                                                            │
 │    214 +         best_truth_idx[best_prior_idx[j]] = j                                                          │
 │    215 +     matches = truths[best_truth_idx]                                                                   │
 │    216 +     conf = labels[best_truth_idx] + 1                                                                  │
 │    217 +     conf[best_truth_overlap < threshold] = 0                                                           │
 │    218 +     loc = encode(matches, priors, variances)                                                           │
 │    219 +     return loc, conf                                                                                   │
 │    220 +                                                                                                        │
 │    221 + def encode(matched, priors, variances):                                                                │
 │    222 +     g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]                                       │
 │    223 +     g_cxcy /= (variances[0] * priors[:, 2:])                                                           │
 │    224 +     g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]                                           │
 │    225 +     g_wh = torch.log(g_wh) / variances[1]                                                              │
 │    226 +     return torch.cat([g_cxcy, g_wh], 1)                                                                │
 │    227 +                                                                                                        │
 │    228 + class MultiBoxLoss(nn.Module):                                                                         │
 │    229 +     def __init__(self, num_classes, overlap_thresh, prior_for_matching, bkg_label, neg_mining,         │
 │        neg_pos, neg_overlap, encode_target):                                                                    │
 │    230 +         super(MultiBoxLoss, self).__init__()                                                           │
 │    231 +         self.num_classes = num_classes                                                                 │
 │    232 +         self.threshold = overlap_thresh                                                                │
 │    233 +         self.background_label = bkg_label                                                              │
 │    234 +         self.encode_target = encode_target                                                             │
 │    235 +         self.use_prior_for_matching = prior_for_matching                                               │
 │    236 +         self.do_neg_mining = neg_mining                                                                │
 │    237 +         self.negpos_ratio = neg_pos                                                                    │
 │    238 +         self.neg_overlap = neg_overlap                                                                 │
 │    239 +         self.variance = PRIOR_BOX_CONFIG['variance']                                                   │
 │    240 +                                                                                                        │
 │    241 +     def forward(self, predictions, targets):                                                           │
 │    242 +         loc_data, conf_data, priors = predictions                                                      │
 │    243 +         num = loc_data.size(0)                                                                         │
 │    244 +         priors = priors[:loc_data.size(1), :]                                                          │
 │    245 +         num_priors = (priors.size(0))                                                                  │
 │    246 +                                                                                                        │
 │    247 +         loc_t = torch.Tensor(num, num_priors, 4)                                                       │
 │    248 +         conf_t = torch.LongTensor(num, num_priors)                                                     │
 │    249 +         for idx in range(num):                                                                         │
 │    250 +             truths = targets[idx][:, :-1]                                                              │
 │    251 +             labels = targets[idx][:, -1]                                                               │
 │    252 +             defaults = priors                                                                          │
 │    253 +             match(self.threshold, truths, defaults, self.variance, labels, loc_t, conf_t, idx)         │
 │    254 +         loc_t = loc_t.to(loc_data.device)                                                              │
 │    255 +         conf_t = conf_t.to(conf_data.device)                                                           │
 │    256 +                                                                                                        │
 │    257 +         pos = conf_t > 0                                                                               │
 │    258 +         num_pos = pos.sum(dim=1, keepdim=True)                                                         │
 │    259 +                                                                                                        │
 │    260 +         pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)                                         │
 │    261 +         loc_p = loc_data[pos_idx].view(-1, 4)                                                          │
 │    262 +         loc_t = loc_t[pos_idx].view(-1, 4)                                                             │
 │    263 +         loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')                                       │
 │    264 +                                                                                                        │
 │    265 +         batch_conf = conf_data.view(-1, self.num_classes)                                              │
 │    266 +         loss_c = F.cross_entropy(batch_conf, conf_t.view(-1), reduction='none')                        │
 │    267 +         loss_c = loss_c.view(num, -1)                                                                  │
 │    268 +         loss_c[pos] = 0                                                                                │
 │    269 +                                                                                                        │
 │    270 +         _, loss_idx = loss_c.sort(1, descending=True)                                                  │
 │    271 +         _, idx_rank = loss_idx.sort(1)                                                                 │
 │    272 +         num_pos = pos.long().sum(1, keepdim=True)                                                      │
 │    273 +         num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)                            │
 │    274 +         neg = idx_rank < num_neg.expand_as(idx_rank)                                                   │
 │    275 +                                                                                                        │
 │    276 +         pos_idx = pos.unsqueeze(2).expand_as(conf_data)                                                │
 │    277 +         neg_idx = neg.unsqueeze(2).expand_as(conf_data)                                                │
 │    278 +         conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)                         │
 │    279 +         targets_weighted = conf_t[(pos+neg).gt(0)]                                                     │
 │    280 +         loss_c = F.cross_entropy(conf_p, targets_weighted, reduction='sum')                            │
 │    281 +                                                                                                        │
 │    282 +         N = num_pos.sum().float()                                                                      │
 │    283 +         loss_l /= N                                                                                    │
 │    284 +         loss_c /= N                                                                                    │
 │    285 +         return loss_l, loss_c                                                                          │
 │    286 +                                                                                                        │
 │    287 + if __name__ == '__main__':                                                                             │
 │    288 +     transform = SSDAugmentation()                                                                      │
 │    289       train_txt_path = os.path.join(DATA_PATH, 'wider_face_split', 'wider_face_train_bbx_gt.txt')        │
 │    290       train_image_dir = os.path.join(DATA_PATH, 'WIDER_train', 'images')                                 │
 │    233 -                                                                                                        │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    291       train_dataset = WiderFaceDataset(txt_path=train_txt_path, image_dir=train_image_dir,               │
 │          transform=transform)                                                                                   │
 │    235 -                                                                                                        │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    292       train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False,    │
 │          collate_fn=collate_fn)                                                                                 │
 │    293                                                                                                          │
 │    238 -     print(f'배치 크기: {BATCH_SIZE}')                                                                  │
 │    239 -     images, _, _ = next(iter(train_loader))                                                            │
 │    240 -     print(f'이미지 배치 텐서 모양: {images.shape}')                                                    │
 │    241 -                                                                                                        │
 │    242 -     # --- 최종 SSD 모델 생성 및 확인 ---                                                               │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    294 +     # --- 모델 생성 ---                                                                                │
 │    295       vgg_layers = vgg_base()                                                                            │
 │    296       extra_layers = add_extras()                                                                        │
 │    297       head_in_channels = [512, 1024, 512, 256, 256, 256]                                                 │
 │    298       head_layers = multibox_head(head_in_channels, NUM_CLASSES)                                         │
 │    247 -                                                                                                        │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    299       ssd_model = SSD(vgg_layers, extra_layers, head_layers, NUM_CLASSES)                                │
 │    300 +     ssd_model.eval() # 테스트를 위해 eval 모드로                                                       │
 │    301                                                                                                          │
 │    250 -     # 모델에 이미지 배치를 통과시켜 최종 출력 확인                                                     │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    302 +     # --- Loss 함수 생성 및 확인 ---                                                                   │
 │    303 +     criterion = MultiBoxLoss(NUM_CLASSES, 0.5, True, 0, True, 3, 0.5, False)                           │
 │    304 +                                                                                                        │
 │    305 +     # 데이터로더에서 한 배치 가져오기                                                                  │
 │    306 +     images, boxes, labels = next(iter(train_loader))                                                   │
 │    307 +                                                                                                        │
 │    308 +     # 모델 예측                                                                                        │
 │    309       loc, conf, priors = ssd_model(images)                                                              │
 │    310                                                                                                          │
 │    253 -     print('\n--- 최종 SSD 모델 출력 확인 ---')                                                         │
 │    254 -     print(f'priors 모양: {priors.shape}')                                                              │
 │    255 -     print(f'loc(위치) 예측 모양: {loc.shape}')                                                         │
 │    256 -     print(f'conf(신뢰도) 예측 모양: {conf.shape}')                                                     │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    311 +     # Loss 계산을 위해 target 준비                                                                     │
 │    312 +     targets = [torch.cat((box, lab.unsqueeze(1).float()), 1) for box, lab in zip(boxes, labels)]       │
 │    313 +                                                                                                        │
 │    314 +     # Loss 계산                                                                                        │
 │    315 +     loss_l, loss_c = criterion((loc, conf, priors), targets)                                           │
 │    316 +                                                                                                        │
 │    317 +     print('\n--- MultiBoxLoss 출력 확인 ---')                                                          │
 │    318 +     print(f'Location Loss: {loss_l.item()}')                                                           │
 │    319 +     print(f'Confidence Loss: {loss_c.item()}')                                                         │
 │    320 +     total_loss = loss_l + loss_c                                                                       │
 │    321 +     print(f'Total Loss: {total_loss.item()}')                                                          │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 네, MultiBoxLoss와 관련 유틸리티 함수들을 포함한 전체 코드를 face_detector_script2.py 파일에 성공적으로 업데이트했습니다.


  이제 마지막으로 스크립트를 실행하여, 완성된 손실 함수가 모델의 예측값과 정답 데이터를 이용해 최종 손실을 올바르게
  계산하는지 확인해 보겠습니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  Shell python "D:/project/Aiffel/face detector/face_detector_script2.py"                                      │
 │                                                                                                                 │
                                                                          │
                    │ect\Aiffel\face detector\face_detector_script2.py", line 315, in <module>
                                                │priors), targets)
                                                │^^^^^^^^^^^^^^^^^
 │      File                                                                                                       │
 │    "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py", line  │
                                                                                 │
                                                                 │
                                                                 │
 │      File                                                                                                       │
 │    "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py", line  │
                                                                                         │
                                                                    │
                     │ct\Aiffel\face detector\face_detector_script2.py", line 253, in forward
                      │reshold, truths, defaults, self.variance, labels, loc_t, conf_t, idx)
                      │^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 │    TypeError: match() takes 5 positional arguments but 8 were given                                             │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 네, 또 오류가 발생했습니다. 죄송합니다. 원인을 파악하고 즉시 수정하겠습니다.

  오류 원인:

  `
  TypeError: match() takes 5 positional arguments but 8 were given
  `


  이 오류는 MultiBoxLoss의 forward 함수 안에서 match라는 헬퍼 함수를 호출할 때 발생했습니다. match 함수는 5개의
  인자(parameter)를 받도록 정의되어 있는데, 제가 실수로 8개의 인자를 전달하여 함수 시그니처가 맞지 않게 되었습니다.

  해결:


  MultiBoxLoss의 forward 함수 내에서 match 함수를 올바르게 호출하도록 수정하겠습니다. match 함수가 계산한 결과를
  반환하면, forward 함수가 그 값을 받아 처리하는 방식으로 코드를 변경합니다.


  다시 한번 write_file을 사용하여 스크립트 전체를 올바른 코드로 덮어쓰겠습니다. 이번에는 반드시 성공적으로 실행하여
  결과를 보여드리겠습니다.
 ╭───────────────────────────────────────╮
 │ ✓  ReadFile face_detector_script2.py  │
 ╰───────────────────────────────────────╯
✦ 네, 파일 내용을 다시 확인했습니다. match 함수를 잘못 호출하여 TypeError가 발생한 버전이 맞습니다.

  이제 이 오류를 포함한 몇 가지 자잘한 버그들을 모두 수정한 최종 완성본 코드로 face_detector_script2.py 파일을
  덮어쓰겠습니다.

  주요 수정 내용:


   1. `MultiBoxLoss.forward`: match 함수를 올바른 인자 개수로 호출하고, 그 반환값을 받아 처리하도록 수정 (핵심 오류 수정)
  이제 정말 최종본입니다. 이 코드를 파일에 쓰고 실행하여 결과를 확인하겠습니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  WriteFile Writing to face_detector_script2.py                                                                │
 │                                                                                                                 │
 │    ... first 56 lines hidden ...                                                                                │
 │     58 +             T.Resize(self.size, antialias=True),                                                       │
 │     59 +             T.Normalize(mean=self.mean, std=self.std)                                                  │
 │     60 +         ])                                                                                             │
 │     61 +     def __call__(self, img, boxes, labels):                                                            │
 │     62 +         width, height = img.size                                                                       │
 │     63 +         boxes[:, 0] /= width                                                                           │
 │     64 +         boxes[:, 1] /= height                                                                          │
 │     65 +         boxes[:, 2] /= width                                                                           │
 │     66 +         boxes[:, 3] /= height                                                                          │
 │     67 +         return self.transform(img), torch.from_numpy(boxes), torch.from_numpy(labels)                  │
 │     68 +                                                                                                        │
 │     69   class WiderFaceDataset(torch.utils.data.Dataset):                                                      │
 │     59 -     # ... (이전과 동일)                                                                                │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     70       def __init__(self, txt_path, image_dir, transform=None):                                           │
 │     71           self.infos = parse_widerface(txt_path)                                                         │
 │     72           self.image_dir = image_dir                                                                     │
 │     77           img_info = self.infos[idx]                                                                     │
 │     78           image_path = os.path.join(self.image_dir, img_info[0])                                         │
 │     79           boxes = np.array(img_info[1], dtype=np.float32)                                                │
 │     80 +         from PIL import Image                                                                          │
 │     81           image = Image.open(image_path).convert('RGB')                                                  │
 │     82           labels = np.ones(boxes.shape[0], dtype=np.int64)                                               │
 │     83           if self.transform:                                                                             │
 │     84               image, boxes, labels = self.transform(image, boxes, labels)                                │
 │     85           return image, boxes, labels                                                                    │
 │     75 -         return {'image': image, 'boxes': boxes, 'labels': labels}                                      │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     86                                                                                                          │
 │     77 - class SSDAugmentation:                                                                                 │
 │     78 -     def __init__(self, size=(320, 256), mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)):        │
 │     79 -         self.mean = mean                                                                               │
 │     80 -         self.std = std                                                                                 │
 │     81 -         self.size = size                                                                               │
 │     82 -         self.augment = T.Compose([                                                                     │
 │     83 -             T.ToTensor(),                                                                              │
 │     84 -             T.Resize(self.size),                                                                       │
 │     85 -             T.Normalize(mean=self.mean, std=self.std)                                                  │
 │     86 -         ])                                                                                             │
 │     87 -     def __call__(self, img, boxes, labels):                                                            │
 │     88 -         return self.augment(img), torch.from_numpy(boxes), torch.from_numpy(labels)                    │
 │     89 -                                                                                                        │
 │     87   def collate_fn(batch):                                                                                 │
 │     88       images = torch.stack([item[0] for item in batch], dim=0)                                           │
 │     89       boxes = [item[1] for item in batch]                                                                │
 │     93   # =====================================================================                                │
 │     94   # 2. SSD 모델 아키텍처                                                                                 │
 │     95   # =====================================================================                                │
 │     99 - # vgg_base, add_extras, PriorBox, multibox_head, SSD 클래스 (이전과 동일)                              │
 │     96   def vgg_base():                                                                                        │
 │     97       layers = []                                                                                        │
 │     98       in_channels = 3                                                                                    │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    175   # =====================================================================                                │
 │    176   # 3. 손실 함수 (MultiBoxLoss)                                                                          │
 │    177   # =====================================================================                                │
 │    182 -                                                                                                        │
 │    178   def point_form(boxes):                                                                                 │
 │    179       return torch.cat((boxes[:, :2] - boxes[:, 2:]/2, boxes[:, :2] + boxes[:, 2:]/2), 1)                │
 │    185 -                                                                                                        │
 │    186 - def center_size(boxes):                                                                                │
 │    187 -     return torch.cat(((boxes[:, 2:] + boxes[:, :2])/2, boxes[:, 2:] - boxes[:, :2]), 1)                │
 │    188 -                                                                                                        │
 │    180 + def jaccard(box_a, box_b):                                                                             │
 │    181       def intersect(box_a, box_b):                                                                       │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    190 -     A = box_a.size(0)                                                                                  │
 │    191 -     B = box_b.size(0)                                                                                  │
 │    192 -     max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2), box_b[:, 2:].unsqueeze(0).expand(A,  │
 │        B, 2))                                                                                                   │
 │    193 -     min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2), box_b[:, :2].unsqueeze(0).expand(A,  │
 │        B, 2))                                                                                                   │
 │    182 +         max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(box_a.size(0), box_b.size(0), 2),          │
 │        box_b[:, 2:].unsqueeze(0).expand(box_a.size(0), box_b.size(0), 2))                                       │
 │    183 +         min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(box_a.size(0), box_b.size(0), 2),          │
 │        box_b[:, :2].unsqueeze(0).expand(box_a.size(0), box_b.size(0), 2))                                       │
 │    184           inter = torch.clamp(max_xy - min_xy, min=0)                                                    │
 │    185           return inter[:, :, 0] * inter[:, :, 1]                                                         │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    196 -                                                                                                        │
 │    197 - def jaccard(box_a, box_b):                                                                             │
 │    186       inter = intersect(box_a, box_b)                                                                    │
 │    187       area_a = ((box_a[:, 2]-box_a[:, 0]) * (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)     │
 │    188       area_b = ((box_b[:, 2]-box_b[:, 0]) * (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)     │
 │    189       union = area_a + area_b - inter                                                                    │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    202 -     return inter / union                                                                               │
 │    190 +     return inter / (union + 1e-6)                                                                      │
 │    191                                                                                                          │
 │    192 + def encode(matched, priors, variances):                                                                │
 │    193 +     g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]                                       │
 │    194 +     g_cxcy /= (variances[0] * priors[:, 2:])                                                           │
 │    195 +     g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]                                           │
 │    196 +     g_wh = torch.log(g_wh) / variances[1]                                                              │
 │    197 +     return torch.cat([g_cxcy, g_wh], 1)                                                                │
 │    198 +                                                                                                        │
 │    199   def match(threshold, truths, priors, variances, labels):                                               │
 │    200       overlaps = jaccard(truths, point_form(priors))                                                     │
 │    201       best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True)                                 │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    213       loc = encode(matches, priors, variances)                                                           │
 │    214       return loc, conf                                                                                   │
 │    215                                                                                                          │
 │    221 - def encode(matched, priors, variances):                                                                │
 │    222 -     g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]                                       │
 │    223 -     g_cxcy /= (variances[0] * priors[:, 2:])                                                           │
 │    224 -     g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]                                           │
 │    225 -     g_wh = torch.log(g_wh) / variances[1]                                                              │
 │    226 -     return torch.cat([g_cxcy, g_wh], 1)                                                                │
 │    227 -                                                                                                        │
 │    216   class MultiBoxLoss(nn.Module):                                                                         │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    229 -     def __init__(self, num_classes, overlap_thresh, prior_for_matching, bkg_label, neg_mining,         │
 │        neg_pos, neg_overlap, encode_target):                                                                    │
 │    217 +     def __init__(self, num_classes=NUM_CLASSES, overlap_thresh=0.5, neg_pos=3):                        │
 │    218           super(MultiBoxLoss, self).__init__()                                                           │
 │    219           self.num_classes = num_classes                                                                 │
 │    220           self.threshold = overlap_thresh                                                                │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    233 -         self.background_label = bkg_label                                                              │
 │    234 -         self.encode_target = encode_target                                                             │
 │    235 -         self.use_prior_for_matching = prior_for_matching                                               │
 │    236 -         self.do_neg_mining = neg_mining                                                                │
 │    221           self.negpos_ratio = neg_pos                                                                    │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    238 -         self.neg_overlap = neg_overlap                                                                 │
 │    222           self.variance = PRIOR_BOX_CONFIG['variance']                                                   │
 │    223                                                                                                          │
 │    224       def forward(self, predictions, targets):                                                           │
 │    225           loc_data, conf_data, priors = predictions                                                      │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    243 -         num = loc_data.size(0)                                                                         │
 │    244 -         priors = priors[:loc_data.size(1), :]                                                          │
 │    245 -         num_priors = (priors.size(0))                                                                  │
 │    226 +         num_batches = loc_data.size(0)                                                                 │
 │    227 +         num_priors = priors.size(0)                                                                    │
 │    228                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    247 -         loc_t = torch.Tensor(num, num_priors, 4)                                                       │
 │    248 -         conf_t = torch.LongTensor(num, num_priors)                                                     │
 │    249 -         for idx in range(num):                                                                         │
 │    250 -             truths = targets[idx][:, :-1]                                                              │
 │    251 -             labels = targets[idx][:, -1]                                                               │
 │    252 -             defaults = priors                                                                          │
 │    253 -             match(self.threshold, truths, defaults, self.variance, labels, loc_t, conf_t, idx)         │
 │    229 +         loc_t = torch.Tensor(num_batches, num_priors, 4)                                               │
 │    230 +         conf_t = torch.LongTensor(num_batches, num_priors)                                             │
 │    231 +                                                                                                        │
 │    232 +         for idx in range(num_batches):                                                                 │
 │    233 +             truths = targets[idx][:, :-1].data                                                         │
 │    234 +             labels = targets[idx][:, -1].data                                                          │
 │    235 +             if truths.numel() == 0:                                                                    │
 │    236 +                 conf_t[idx].fill_(0)                                                                   │
 │    237 +                 continue                                                                               │
 │    238 +                                                                                                        │
 │    239 +             matched_loc_t, matched_conf_t = match(self.threshold, truths, priors.data, self.variance,  │
 │        labels)                                                                                                  │
 │    240 +             loc_t[idx] = matched_loc_t                                                                 │
 │    241 +             conf_t[idx] = matched_conf_t                                                               │
 │    242 +                                                                                                        │
 │    243           loc_t = loc_t.to(loc_data.device)                                                              │
 │    244           conf_t = conf_t.to(conf_data.device)                                                           │
 │    245                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    253                                                                                                          │
 │    254           batch_conf = conf_data.view(-1, self.num_classes)                                              │
 │    255           loss_c = F.cross_entropy(batch_conf, conf_t.view(-1), reduction='none')                        │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    267 -         loss_c = loss_c.view(num, -1)                                                                  │
 │    256 +         loss_c = loss_c.view(num_batches, -1)                                                          │
 │    257           loss_c[pos] = 0                                                                                │
 │    258                                                                                                          │
 │    259           _, loss_idx = loss_c.sort(1, descending=True)                                                  │
 │    260           _, idx_rank = loss_idx.sort(1)                                                                 │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    272 -         num_pos = pos.long().sum(1, keepdim=True)                                                      │
 │    273 -         num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)                            │
 │    261 +         num_pos_per_batch = pos.long().sum(1, keepdim=True)                                            │
 │    262 +         num_neg = torch.clamp(self.negpos_ratio*num_pos_per_batch, max=num_priors-1)                   │
 │    263           neg = idx_rank < num_neg.expand_as(idx_rank)                                                   │
 │    264                                                                                                          │
 │    265           pos_idx = pos.unsqueeze(2).expand_as(conf_data)                                                │
 │    269           loss_c = F.cross_entropy(conf_p, targets_weighted, reduction='sum')                            │
 │    270                                                                                                          │
 │    271           N = num_pos.sum().float()                                                                      │
 │    272 +         if N == 0: N = num_batches # 정답 박스가 없는 경우 loss가 0이 되는 것을 방지                   │
 │    273           loss_l /= N                                                                                    │
 │    274           loss_c /= N                                                                                    │
 │    275           return loss_l, loss_c                                                                          │
 │    279       train_txt_path = os.path.join(DATA_PATH, 'wider_face_split', 'wider_face_train_bbx_gt.txt')        │
 │    280       train_image_dir = os.path.join(DATA_PATH, 'WIDER_train', 'images')                                 │
 │    281       train_dataset = WiderFaceDataset(txt_path=train_txt_path, image_dir=train_image_dir,               │
 │          transform=transform)                                                                                   │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    292 -     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False,    │
 │        collate_fn=collate_fn)                                                                                   │
 │    282 +     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,     │
 │        collate_fn=collate_fn)                                                                                   │
 │    283                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    294 -     # --- 모델 생성 ---                                                                                │
 │    295 -     vgg_layers = vgg_base()                                                                            │
 │    296 -     extra_layers = add_extras()                                                                        │
 │    297 -     head_in_channels = [512, 1024, 512, 256, 256, 256]                                                 │
 │    298 -     head_layers = multibox_head(head_in_channels, NUM_CLASSES)                                         │
 │    299 -     ssd_model = SSD(vgg_layers, extra_layers, head_layers, NUM_CLASSES)                                │
 │    300 -     ssd_model.eval() # 테스트를 위해 eval 모드로                                                       │
 │    284 +     ssd_model = SSD(vgg_base(), add_extras(), multibox_head([512, 1024, 512, 256, 256, 256],           │
 │        NUM_CLASSES), NUM_CLASSES)                                                                               │
 │    285 +     ssd_model.eval()                                                                                   │
 │    286                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    302 -     # --- Loss 함수 생성 및 확인 ---                                                                   │
 │    303 -     criterion = MultiBoxLoss(NUM_CLASSES, 0.5, True, 0, True, 3, 0.5, False)                           │
 │    287 +     criterion = MultiBoxLoss()                                                                         │
 │    288                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    305 -     # 데이터로더에서 한 배치 가져오기                                                                  │
 │    289       images, boxes, labels = next(iter(train_loader))                                                   │
 │    290                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    308 -     # 모델 예측                                                                                        │
 │    291       loc, conf, priors = ssd_model(images)                                                              │
 │    292                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    311 -     # Loss 계산을 위해 target 준비                                                                     │
 │    293       targets = [torch.cat((box, lab.unsqueeze(1).float()), 1) for box, lab in zip(boxes, labels)]       │
 │    294                                                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    314 -     # Loss 계산                                                                                        │
 │    295       loss_l, loss_c = criterion((loc, conf, priors), targets)                                           │
 │    296                                                                                                          │
 │    297       print('\n--- MultiBoxLoss 출력 확인 ---')                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    318 -     print(f'Location Loss: {loss_l.item()}')                                                           │
 │    319 -     print(f'Confidence Loss: {loss_c.item()}')                                                         │
 │    298 +     print(f'Location Loss: {loss_l.item():.4f}')                                                       │
 │    299 +     print(f'Confidence Loss: {loss_c.item():.4f}')                                                     │
 │    300       total_loss = loss_l + loss_c                                                                       │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    321 -     print(f'Total Loss: {total_loss.item()}')                                                          │
 │    301 +     print(f'Total Loss: {total_loss.item():.4f}')                                                      │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 네, MultiBoxLoss와 관련 버그들을 수정한 최종 코드를 파일에 덮어썼습니다.


  이제 마지막으로 스크립트를 실행하여, 손실 함수가 정말로 오류 없이 최종 Loss 값을 계산하는지 확인하겠습니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  Shell python "D:/project/Aiffel/face detector/face_detector_script2.py"                                      │
 │                                                                                                                 │
                                                                          │
                    │ect\Aiffel\face detector\face_detector_script2.py", line 295, in <module>
                                                │priors), targets)
                                                │^^^^^^^^^^^^^^^^^
 │      File                                                                                                       │
 │    "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py", line  │
                                                                                 │
                                                                 │
                                                                 │
 │      File                                                                                                       │
 │    "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py", line  │
                                                                                         │
                                                                    │
                     │ct\Aiffel\face detector\face_detector_script2.py", line 255, in forward
                                 │(batch_conf, conf_t.view(-1), reduction='none')
 │      File "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\functional.py",    │
                                                                                 │
                                                                 │
                                                                 │
                                                                                              │
                                                                                              │
                                                                                         │
                                                                                    │
                                                                                    │
                                                                                                       │
                                                                                                       │
 │    IndexError: Target 2 is out of bounds.                                                                       │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 네, 또다시 오류가 발생했습니다. 이번에는 IndexError입니다. 계속된 오류에 정말 죄송한 마음입니다. 원인을 정확히 파악하고
  수정하겠습니다.

  오류 원인:

  `
  IndexError: Target 2 is out of bounds.
  `


  이 오류는 손실 함수 CrossEntropyLoss에 전달된 정답 레이블에 2라는 값이 포함되어 있다는 의미입니다. 저희 모델의 클래스는
  0 (배경)과 1 (얼굴), 총 2개이므로 레이블은 0 또는 1이어야 합니다.


  버그 위치:
  match 함수 안의 conf = labels[best_truth_idx] + 1 이 부분이 문제입니다.
  데이터셋에서 '얼굴'의 레이블은 1인데, 여기에 1을 더하여 정답 레이블을 2로 잘못 만들고 있었습니다.

  해결:
  match 함수에서 + 1 부분을 제거하여, '얼굴'의 정답 레이블이 1로 올바르게 유지되도록 수정하겠습니다.


  이번이 정말 마지막이 되도록, 코드를 수정하여 파일 전체를 다시 쓰고 최종 실행하겠습니다. 너그러이 양해해 주셔서
  감사합니다.
 ╭───────────────────────────────────────╮
 │ ✓  ReadFile face_detector_script2.py  │
 ╰───────────────────────────────────────╯
✦ 네, IndexError가 발생하는 코드 내용을 다시 확인했습니다.


  이제 match 함수에서 레이블을 잘못 계산하는 부분을 수정한 최종 코드로 파일 전체를 덮어쓰겠습니다. 그리고 즉시 실행하여,
  이번에야말로 손실 함수가 올바르게 작동하는 것을 보여드리겠습니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  WriteFile Writing to face_detector_script2.py                                                                │
 │                                                                                                                 │
 │     60       ])                                                                                                 │
 │     61   def __call__(self, img, boxes, labels):                                                                │
 │     62       width, height = img.size                                                                           │
 │     63 +     if boxes.shape[0] > 0:                                                                             │
 │     64           boxes[:, 0] /= width                                                                           │
 │     65           boxes[:, 1] /= height                                                                          │
 │     66           boxes[:, 2] /= width                                                                           │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    209   for j in range(best_prior_idx.size(0)):                                                                │
 │    210       best_truth_idx[best_prior_idx[j]] = j                                                              │
 │    211   matches = truths[best_truth_idx]                                                                       │
 │    211 - conf = labels[best_truth_idx] + 1                                                                      │
 │    212 + conf = labels[best_truth_idx] # BUG FIX: removed +1                                                    │
 │    213   conf[best_truth_overlap < threshold] = 0                                                               │
 │    214   loc = encode(matches, priors, variances)                                                               │
 │    215   return loc, conf                                                                                       │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    235           labels = targets[idx][:, -1].data                                                              │
 │    236           if truths.numel() == 0:                                                                        │
 │    237               conf_t[idx].fill_(0)                                                                       │
 │    238 +             loc_t[idx].fill_(0)                                                                        │
 │    239               continue                                                                                   │
 │    240                                                                                                          │
 │    241           matched_loc_t, matched_conf_t = match(self.threshold, truths, priors.data, self.variance,      │
 │          labels)                                                                                                │
 │    246       conf_t = conf_t.to(conf_data.device)                                                               │
 │    247                                                                                                          │
 │    248       pos = conf_t > 0                                                                                   │
 │    247 -     num_pos = pos.sum(dim=1, keepdim=True)                                                             │
 │    249                                                                                                          │
 │    250       pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)                                             │
 │    251       loc_p = loc_data[pos_idx].view(-1, 4)                                                              │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    269       targets_weighted = conf_t[(pos+neg).gt(0)]                                                         │
 │    270       loss_c = F.cross_entropy(conf_p, targets_weighted, reduction='sum')                                │
 │    271                                                                                                          │
 │    271 -     N = num_pos.sum().float()                                                                          │
 │    272 -     if N == 0: N = num_batches # 정답 박스가 없는 경우 loss가 0이 되는 것을 방지                       │
 │    272 +     N = pos.long().sum(1, keepdim=True).sum().float()                                                  │
 │    273 +     if N == 0: N = num_batches                                                                         │
 │    274       loss_l /= N                                                                                        │
 │    275       loss_c /= N                                                                                        │
 │    276       return loss_l, loss_c                                                                              │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 네, IndexError를 유발한 match 함수를 수정한 최종 코드를 파일에 덮어썼습니다.


  이제 마지막으로 스크립트를 실행하여, 모든 오류가 해결되었는지 그리고 손실 함수가 정상적으로 값을 계산하는지
  확인하겠습니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  Shell python "D:/project/Aiffel/face detector/face_detector_script2.py"                                      │
 │                                                                                                                 │
                                                                              │
                                                                                       │
                                                                                     │
 │    Total Loss: 8.1126                                                                                           │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 드디어 모든 오류를 해결하고 손실 함수까지 구현을 완료했습니다!

  최종 실행 결과:

  `
  --- MultiBoxLoss 출력 확인 ---
  Location Loss: 5.2858
  Confidence Loss: 2.8268
  Total Loss: 8.1126
  `

  결론:


   * 성공: 보시는 바와 같이, Location Loss와 Confidence Loss가 성공적으로 계산되었습니다. 이로써 우리는 학습에 필요한 모든
     구성 요소(데이터로더, 모델, 손실 함수)를 갖추게 되었습니다.
  4단계: 학습 루프(Training Loop) 구현

  지금까지 만든 모든 것을 하나로 합쳐, 모델이 데이터셋을 반복적으로 학습하고 똑똑해지도록 만드는 실제 "훈련" 코드를
  작성하는 단계입니다.


  이 단계까지 진행하면, 우리는 WIDER FACE 데이터셋으로 학습된 Face Detector 모델을 얻게 됩니다. 마지막 단계를 진행할까요?

╭────────────╮
│  > 진행해  │
╰────────────╯

✦ 네, 마지막 4단계: 학습 루프(Training Loop) 구현을 시작하겠습니다.


  지금까지 만든 데이터로더, 모델, 손실 함수를 모두 사용하여, 모델을 실제로 훈련시키는 코드를 작성하겠습니다. 훈련 중에는
  각 단계별 손실(Loss)을 출력하여 학습 과정을 확인할 수 있도록 하고, 한 에폭(epoch)이 끝날 때마다 학습된 모델의 가중치를
  파일로 저장하여 진행 상황을 보존하도록 하겠습니다.


  전체 스크립트를 최종 훈련 코드로 업데이트하겠습니다.
 ╭───────────────────────────────────────╮
 │ ✓  ReadFile face_detector_script2.py  │
 ╰───────────────────────────────────────╯
✦ 네, 확인했습니다. 이제 마지막 단계인 학습 루프를 구현하여, 지금까지 만든 모든 구성 요소가 함께 작동하여 모델을
  훈련시키도록 하겠습니다.


  전체 코드를 최종 훈련 스크립트로 업데이트하고, 1 에폭(Epoch)만 실행하여 학습이 정상적으로 시작되는지 확인하겠습니다.
  학습이 시작되면 반복적으로 손실(Loss)이 출력될 것입니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  WriteFile Writing to face_detector_script2.py                                                                │
 │                                                                                                                 │
 │      2   import torch.nn as nn                                                                                  │
 │      3   import torch.nn.functional as F                                                                        │
 │      4   import torchvision.transforms as T                                                                     │
 │      5 + import torch.optim as optim                                                                            │
 │      6                                                                                                          │
 │      6 - import os, cv2, time                                                                                   │
 │      7 + import os, time                                                                                        │
 │      8   import numpy as np                                                                                     │
 │      9   import math                                                                                            │
 │     10   from itertools import product                                                                          │
 │     11                                                                                                          │
 │     12 + # =====================================================================                                │
 │     13 + # 0. 설정 (Configuration)                                                                              │
 │     14 + # =====================================================================                                │
 │     15   PROJECT_PATH = 'D:/project/Aiffel/face detector'                                                       │
 │     16   DATA_PATH = os.path.join(PROJECT_PATH, 'widerface')                                                    │
 │     17 + CHECKPOINT_PATH = os.path.join(PROJECT_PATH, 'checkpoints')                                            │
 │     18                                                                                                          │
 │     14 - BATCH_SIZE = 4                                                                                         │
 │     19 + # 학습 하이퍼파라미터                                                                                  │
 │     20 + EPOCHS = 1 # 데모를 위해 1 에폭만 실행                                                                 │
 │     21 + BATCH_SIZE = 8                                                                                         │
 │     22 + LR = 1e-3                                                                                              │
 │     23 + MOMENTUM = 0.9                                                                                         │
 │     24 + WEIGHT_DECAY = 5e-4                                                                                    │
 │     25 +                                                                                                        │
 │     26   IMAGE_WIDTH = 320                                                                                      │
 │     27   IMAGE_HEIGHT = 256                                                                                     │
 │     28   IMAGE_LABELS = ['background', 'face']                                                                  │
 │     31   # =====================================================================                                │
 │     32   # 1. 데이터 준비 (Data Preparation)                                                                    │
 │     33   # =====================================================================                                │
 │     23 -                                                                                                        │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     34   def parse_widerface(file):                                                                             │
 │     35       infos = []                                                                                         │
 │     36       with open(file) as fp:                                                                             │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     60                                                                                                          │
 │     61   class SSDAugmentation:                                                                                 │
 │     62       def __init__(self, size=(IMAGE_WIDTH, IMAGE_HEIGHT), mean=(0.485, 0.456, 0.406), std=(0.229,       │
 │          0.224, 0.225)):                                                                                        │
 │     53 -         self.mean = mean                                                                               │
 │     54 -         self.std = std                                                                                 │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     63           self.size = size                                                                               │
 │     64           self.transform = T.Compose([                                                                   │
 │     65               T.ToTensor(),                                                                              │
 │     66               T.Resize(self.size, antialias=True),                                                       │
 │     59 -             T.Normalize(mean=self.mean, std=self.std)                                                  │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     67 +             T.Normalize(mean=mean, std=std)                                                            │
 │     68           ])                                                                                             │
 │     69       def __call__(self, img, boxes, labels):                                                            │
 │     70 +         from PIL import Image                                                                          │
 │     71           width, height = img.size                                                                       │
 │     72           if boxes.shape[0] > 0:                                                                         │
 │     73               boxes[:, 0] /= width                                                                       │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     84       def __len__(self):                                                                                 │
 │     85           return len(self.infos)                                                                         │
 │     86       def __getitem__(self, idx):                                                                        │
 │     87 +         from PIL import Image                                                                          │
 │     88           img_info = self.infos[idx]                                                                     │
 │     89           image_path = os.path.join(self.image_dir, img_info[0])                                         │
 │     90           boxes = np.array(img_info[1], dtype=np.float32)                                                │
 │     81 -         from PIL import Image                                                                          │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │     91           image = Image.open(image_path).convert('RGB')                                                  │
 │     92           labels = np.ones(boxes.shape[0], dtype=np.int64)                                               │
 │     93           if self.transform:                                                                             │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    218       for j in range(best_prior_idx.size(0)):                                                            │
 │    219           best_truth_idx[best_prior_idx[j]] = j                                                          │
 │    220       matches = truths[best_truth_idx]                                                                   │
 │    212 -     conf = labels[best_truth_idx] # BUG FIX: removed +1                                                │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    221 +     conf = labels[best_truth_idx]                                                                      │
 │    222       conf[best_truth_overlap < threshold] = 0                                                           │
 │    223       loc = encode(matches, priors, variances)                                                           │
 │    224       return loc, conf                                                                                   │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    284           loss_c /= N                                                                                    │
 │    285           return loss_l, loss_c                                                                          │
 │    286                                                                                                          │
 │    287 + # =====================================================================                                │
 │    288 + # 4. 학습 (Training)                                                                                   │
 │    289 + # =====================================================================                                │
 │    290 +                                                                                                        │
 │    291   if __name__ == '__main__':                                                                             │
 │    292 +     # -- 1. 준비 --                                                                                    │
 │    293 +     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                              │
 │    294 +     print(f"Using device: {device}")                                                                   │
 │    295 +                                                                                                        │
 │    296 +     if not os.path.exists(CHECKPOINT_PATH):                                                            │
 │    297 +         os.makedirs(CHECKPOINT_PATH)                                                                   │
 │    298 +                                                                                                        │
 │    299 +     # -- 2. 데이터 로더 --                                                                             │
 │    300       transform = SSDAugmentation()                                                                      │
 │    301       train_txt_path = os.path.join(DATA_PATH, 'wider_face_split', 'wider_face_train_bbx_gt.txt')        │
 │    302       train_image_dir = os.path.join(DATA_PATH, 'WIDER_train', 'images')                                 │
 │    303       train_dataset = WiderFaceDataset(txt_path=train_txt_path, image_dir=train_image_dir,               │
 │          transform=transform)                                                                                   │
 │    283 -     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,     │
 │        collate_fn=collate_fn)                                                                                   │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    304 +     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,     │
 │        collate_fn=collate_fn, num_workers=0)                                                                    │
 │    305 +     print(f"Dataset loaded: {len(train_dataset)} images")                                              │
 │    306                                                                                                          │
 │    285 -     ssd_model = SSD(vgg_base(), add_extras(), multibox_head([512, 1024, 512, 256, 256, 256],           │
 │        NUM_CLASSES), NUM_CLASSES)                                                                               │
 │    286 -     ssd_model.eval()                                                                                   │
 │    287 -                                                                                                        │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    307 +     # -- 3. 모델, 손실함수, 옵티마이저 --                                                              │
 │    308 +     model = SSD(vgg_base(), add_extras(), multibox_head([512, 1024, 512, 256, 256, 256],               │
 │        NUM_CLASSES), NUM_CLASSES).to(device)                                                                    │
 │    309       criterion = MultiBoxLoss()                                                                         │
 │    310 +     optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)     │
 │    311                                                                                                          │
 │    290 -     images, boxes, labels = next(iter(train_loader))                                                   │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    312 +     # -- 4. 학습 루프 --                                                                               │
 │    313 +     print("\n--- Start Training ---")                                                                  │
 │    314 +     for epoch in range(EPOCHS):                                                                        │
 │    315 +         epoch_loss_l = 0                                                                               │
 │    316 +         epoch_loss_c = 0                                                                               │
 │    317 +         start_time = time.time()                                                                       │
 │    318                                                                                                          │
 │    292 -     loc, conf, priors = ssd_model(images)                                                              │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    319 +         for i, (images, boxes, labels) in enumerate(train_loader):                                     │
 │    320 +             model.train()                                                                              │
 │    321                                                                                                          │
 │    294 -     targets = [torch.cat((box, lab.unsqueeze(1).float()), 1) for box, lab in zip(boxes, labels)]       │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    322 +             images = images.to(device)                                                                 │
 │    323 +             targets = [torch.cat((box.to(device), lab.to(device).unsqueeze(1).float()), 1) for box,    │
 │        lab in zip(boxes, labels)]                                                                               │
 │    324                                                                                                          │
 │    296 -     loss_l, loss_c = criterion((loc, conf, priors), targets)                                           │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    325 +             # 순전파                                                                                   │
 │    326 +             loc, conf, priors = model(images)                                                          │
 │    327                                                                                                          │
 │    298 -     print('\n--- MultiBoxLoss 출력 확인 ---')                                                          │
 │    299 -     print(f'Location Loss: {loss_l.item():.4f}')                                                       │
 │    300 -     print(f'Confidence Loss: {loss_c.item():.4f}')                                                     │
 │    301 -     total_loss = loss_l + loss_c                                                                       │
 │    302 -     print(f'Total Loss: {total_loss.item():.4f}')                                                      │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    328 +             # 손실 계산                                                                                │
 │    329 +             loss_l, loss_c = criterion((loc, conf, priors.to(device)), targets)                        │
 │    330 +             loss = loss_l + loss_c                                                                     │
 │    331 +                                                                                                        │
 │    332 +             # 역전파                                                                                   │
 │    333 +             optimizer.zero_grad()                                                                      │
 │    334 +             loss.backward()                                                                            │
 │    335 +             optimizer.step()                                                                           │
 │    336 +                                                                                                        │
 │    337 +             epoch_loss_l += loss_l.item()                                                              │
 │    338 +             epoch_loss_c += loss_c.item()                                                              │
 │    339 +                                                                                                        │
 │    340 +             if (i+1) % 50 == 0:                                                                        │
 │    341 +                 avg_loss_l = epoch_loss_l / (i+1)                                                      │
 │    342 +                 avg_loss_c = epoch_loss_c / (i+1)                                                      │
 │    343 +                 print(f"Epoch [{epoch+1}/{EPOCHS}] | Iter [{i+1}/{len(train_loader)}] | Loc Loss:      │
 │        {avg_loss_l:.4f} | Conf Loss: {avg_loss_c:.4f} | Total Loss: {(avg_loss_l+avg_loss_c):.4f}")             │
 │    344 +                                                                                                        │
 │    345 +         epoch_time = time.time() - start_time                                                          │
 │    346 +         print(f"\nEpoch {epoch+1} Summary | Time: {epoch_time:.2f}s | Avg Loc Loss: {epoch_loss_l/len  │
 │        (train_loader):.4f} | Avg Conf Loss: {epoch_loss_c/len(train_loader):.4f}")                              │
 │    347 +                                                                                                        │
 │    348 +         # 모델 저장                                                                                    │
 │    349 +         save_path = os.path.join(CHECKPOINT_PATH, f'ssd_epoch_{epoch+1}.pth')                          │
 │    350 +         torch.save(model.state_dict(), save_path)                                                      │
 │    351 +         print(f"Epoch {epoch+1} model saved to {save_path}\n")                                         │
 │    352 +                                                                                                        │
 │    353 +     print("--- Training Finished ---")                                                                 │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 네, 최종 학습 코드를 face_detector_script2.py 파일에 덮어썼습니다.


  이제 이 스크립트를 실행하여 전체 프로세스가 작동하고 모델 훈련이 시작되는 것을 보여드리겠습니다. 1 에폭(epoch) 동안의
  학습이 진행되며, 50번의 반복(iteration)마다 현재까지의 평균 손실(loss)이 출력될 것입니다. 마지막으로 학습된 모델이
  checkpoints 폴더에 저장됩니다.


  이제 실행하겠습니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  Shell python "D:/project/Aiffel/face detector/face_detector_script2.py"                                      │
 │                                                                                                                 │
                                                                                          │
                                                                                │
                                                                                                            │
                                                                                      │
                   │Iter [50/1611] | Loc Loss: 8.6628 | Conf Loss: 2.7746 | Total Loss: 11.4374
                  │ Iter [100/1611] | Loc Loss: 8.3674 | Conf Loss: 2.7515 | Total Loss: 11.1188
                  │ Iter [150/1611] | Loc Loss: 7.8471 | Conf Loss: 2.7347 | Total Loss: 10.5819
                  │ Iter [200/1611] | Loc Loss: 7.5202 | Conf Loss: 2.7458 | Total Loss: 10.2660
                         │250/1611] | Loc Loss: inf | Conf Loss: 2.7417 | Total Loss: inf
                         │300/1611] | Loc Loss: inf | Conf Loss: 2.7300 | Total Loss: inf
                         │350/1611] | Loc Loss: inf | Conf Loss: 2.7168 | Total Loss: inf
                         │400/1611] | Loc Loss: inf | Conf Loss: 2.7017 | Total Loss: inf
                         │450/1611] | Loc Loss: inf | Conf Loss: 2.6855 | Total Loss: inf
                         │500/1611] | Loc Loss: inf | Conf Loss: 2.6717 | Total Loss: inf
                         │550/1611] | Loc Loss: inf | Conf Loss: 2.6580 | Total Loss: inf
                         │600/1611] | Loc Loss: inf | Conf Loss: 2.6449 | Total Loss: inf
                                                                          │
                    │ect\Aiffel\face detector\face_detector_script2.py", line 319, in <module>
                                              │merate(train_loader):
                                               │~~~~~^^^^^^^^^^^^^^
 │      File                                                                                                       │
 │    "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py",   │
                                                                                       │
                                                                                │
 │      File                                                                                                       │
 │    "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py",   │
                                                                                     │
                                    │r.fetch(index)  # may raise StopIteration
 │      File                                                                                                       │
 │    "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\_utils\fetch.py"  │
                                                                                         │
                                            │n possibly_batched_index]
                                                                               │
                  │oject\Aiffel\face detector\face_detector_script2.py", line 91, in __getitem__
                                                           │
                                                                          │
 │      File "C:\Users\kicha\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py", line 3513,   │
                                                                                                     │
                                                                      │
 │    FileNotFoundError: [Errno 2] No such file or directory: 'D:/project/Aiffel/face                              │
 │    detector\\widerface\\WIDER_train\\images\\0 0 0 0 0 0 0 0 0 0'                                               │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 학습 중 두 가지 오류가 발생했습니다.


   1. `inf` 손실 (Loss): Loc Loss가 무한대(inf)로 발산했습니다. 이는 encode 함수에서 박스의 너비/높이 비율을 계산하고 로그를
      씌우는 과정에서, 그 비율이 0 또는 음수가 되어 log(0)과 같은 수학적으로 계산할 수 없는 연산이 수행되었기 때문입니다.
   1. `inf` 손실 해결: encode 함수에서 로그를 계산하기 전에, 값이 극단적으로 작아지는 것을 방지하기 위해 아주 작은
      수(1e-6)를 더해주는 안정화 코드를 추가하겠습니다.
  이 수정 사항들을 적용한 최종 코드로 다시 한번 실행하겠습니다. 이번에는 안정적으로 학습이 진행될 것입니다.
 ╭───────────────────────────────────────╮
 │ ✓  ReadFile face_detector_script2.py  │
 ╰───────────────────────────────────────╯
✦ 네, 현재 코드 파일을 확인했습니다.


  이제 말씀드린 두 가지 오류(inf 손실, FileNotFoundError)를 모두 수정한 최종 코드로 파일을 다시 작성하고, 마지막으로
  실행하여 전체 프로젝트를 완성하겠습니다.
 ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
 │ ✓  WriteFile Writing to face_detector_script2.py                                                                │
 │                                                                                                                 │
 │     37           line = fp.readline()                                                                           │
 │     38           while line:                                                                                    │
 │     39               image_file = line.strip()                                                                  │
 │     40 +             if not image_file.endswith('.jpg'): # BUG FIX: 이미지 경로가 아니면 건너뛰기               │
 │     41 +                 line = fp.readline()                                                                   │
 │     42 +                 continue                                                                               │
 │     43 +                                                                                                        │
 │     44               next_line = fp.readline()                                                                  │
 │     45               if not next_line: break                                                                    │
 │     46               try:                                                                                       │
 │    ═══════════════════════════════════════════════════════════════════════════════════════════════════════════  │
 │    207       g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]                                       │
 │    208       g_cxcy /= (variances[0] * priors[:, 2:])                                                           │
 │    209       g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]                                           │
 │    206 -     g_wh = torch.log(g_wh) / variances[1]                                                              │
 │    210 +     g_wh = torch.log(g_wh.clamp(min=1e-6)) / variances[1] # BUG FIX: log(0) 방지                       │
 │    211       return torch.cat([g_cxcy, g_wh], 1)                                                                │
 │    212                                                                                                          │
 │    213   def match(threshold, truths, priors, variances, labels):                                               │
 ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
✦ 네, 두 가지 버그를 모두 수정한 최종 코드를 파일에 반영했습니다.


  이제 정말 마지막으로 스크립트를 실행하여, 우리가 목표했던 대로 Face Detector 모델의 학습이 안정적으로 진행되는 것을
  확인하겠습니다.